In [ ]:
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

import numpy as np
from numpy import random
import pandas as pd
from pandas.plotting import register_matplotlib_converters
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 125)
pd.options.mode.chained_assignment = None

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from math import sqrt

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
register_matplotlib_converters()
import seaborn as sns
sns.set_style('darkgrid')

from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, RepeatedKFold
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, RobustScaler, QuantileTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.pipeline import Pipeline

from keras import layers
from keras import optimizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.utils import plot_model
from keras.wrappers.scikit_learn import KerasRegressor

import pydot
import researchpy as rp

In [ ]:
os.chdir("C:/Users/Tom/Documents/Course/Semester 3/Project/Data Sets/")

In [ ]:
#Importing the Indoor Air Quality sensor data
IAQ_LR = pd.read_csv("IAQ_livingroom.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)
IAQ_KR = pd.read_csv("IAQ_kitchen.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)
IAQ_BR = pd.read_csv("IAQ_bedroom.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)

#Not usinc the dataset below, importing to show a time series plot on why this data won't be used
IAQ_WC = pd.read_csv("IAQ_bathroom.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)

In [ ]:
#Creating datasets to hold just the data for when devices were together for calibration purposes
Cal_LR = IAQ_LR[IAQ_LR["Calibration"] == 1].drop(["Calibration"], axis=1)
Cal_KR = IAQ_KR[IAQ_KR["Calibration"] == 1].drop(["Calibration"], axis=1)
Cal_BR = IAQ_BR[IAQ_BR["Calibration"] == 1].drop(["Calibration"], axis=1)
Cal_WC = IAQ_WC[IAQ_WC["Calibration"] == 1].drop(["Calibration"], axis=1)

#Removing the calibration observations from our sensor data
IAQ_LR = IAQ_LR[IAQ_LR["Calibration"] == 0].drop(["Calibration"], axis=1)
IAQ_KR = IAQ_KR[IAQ_KR["Calibration"] == 0].drop(["Calibration"], axis=1)
IAQ_BR = IAQ_BR[IAQ_BR["Calibration"] == 0].drop(["Calibration"], axis=1)
IAQ_WC = IAQ_WC[IAQ_WC["Calibration"] == 0].drop(["Calibration"], axis=1)


In [ ]:
#Bringing in the Occupant Activity. 
activity_LR = pd.read_csv("activity_livingroom.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)
activity_KR = pd.read_csv("activity_kitchen.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)
activity_BR = pd.read_csv("activity_bedroom.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)

#Since we measured the bathroom data, we're looking if showering, which was measured only in House 2, has any impact on PM2.5
shower = pd.read_csv("activity_bathroom.csv", parse_dates=['Date and Time'], index_col='Date and Time', dayfirst=True)


In [ ]:
display(activity_LR.describe())
display(activity_KR.describe())
display(activity_BR.describe())

In [ ]:
#Joining all activities in the same dataset as they have unique names such as "LR_MaxOccupancy", "BR_MaxOccupancy" and so on.
activity = pd.merge(activity_LR, activity_KR, left_index=True, right_index=True).merge(
    activity_BR, left_index=True, right_index=True).merge(shower, left_index=True, right_index=True)

#activity.to_csv(r'all_activity.csv')


In [ ]:
#Renaming the columns to include LR, KR and BR tags where appropriate

activity.columns = activity.columns.str.replace("_x", "_LR").str.replace("_y", "_KR")

#We included Window_KR:'Window_BR' since Kitchen had no Window feature, so when we replaced _y with _KR, it got the 2nd entry
#of "Window", which was the bedroom Window as "Window_KR"

activity.rename(columns={'MaxOccupancy':'MaxOccupancy_BR',
                          'Door':'Door_BR',
                          'Curtains':'Curtains_BR',
                          'Heating':'Heating_BR',
                          'Vacuuming':'Vacuuming_BR',
                          'Mopping':'Mopping_BR',
                          'Window_KR':'Window_BR'}, inplace=True)

In [ ]:
#To consider time as a feature in our model, we've created a feature storing all 24 hours and one for Time of day

activity['Hour'] = activity.index.strftime('%H').astype(int)
activity["timeOfDay"]=pd.cut(activity.index.hour, [-1, 5, 11, 17, 24], labels=['Night', "Morning", 'Afternoon', 'Evening'])
activity["timeOfDay"] = activity["timeOfDay"].cat.codes

#activity.to_csv(r'treated datasets/all_activity.csv')

In [ ]:
display(activity.info())
display(activity.describe())

# Calibration

In this section we'll analyse the calibration plots for House 1 and 2 for all sensor features. If they're highly correlated, we'll bring their into agreement through their difference in means. If not, they'll be excluded from the model.

In [ ]:
IAQ_KR['adj_PM2.5'] = 0
IAQ_KR['adj_CO2'] = 0
IAQ_KR['adj_TVOC'] = 0
IAQ_KR['adj_Temp'] = 0
IAQ_KR['adj_RH'] = 0
IAQ_KR['adj_AirPres'] = 0

IAQ_LR['adj_PM2.5'] = 0
IAQ_LR['adj_CO2'] = 0
IAQ_LR['adj_TVOC'] = 0
IAQ_LR['adj_Temp'] = 0
IAQ_LR['adj_RH'] = 0
IAQ_LR['adj_AirPres'] = 0

IAQ_BR['adj_PM2.5'] = 0
IAQ_BR['adj_CO2'] = 0
IAQ_BR['adj_TVOC'] = 0
IAQ_BR['adj_Temp'] = 0
IAQ_BR['adj_RH'] = 0
IAQ_BR['adj_AirPres'] = 0

In [ ]:
##### Calibration and Activity plot for House 1 - Reason for why the Bathroom data was excluded

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["PM2.5"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["PM2.5"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["PM2.5"][Cal_BR["House"] == 1], label='Bedroom')
plt.plot(Cal_WC["PM2.5"][Cal_WC["House"] == 1], label='Bathroom')
plt.xlabel('Hours', size=12)
plt.ylabel('PM2.5', size=12)
plt.legend(loc=1, prop={'size': 12})
plt.title("PM2.5 Calibration - House 1")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["PM2.5"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["PM2.5"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["PM2.5"][IAQ_BR["House"] == 1], label='Bedroom')
plt.plot(IAQ_WC["PM2.5"][IAQ_WC["House"] == 1], label='Bathroom')
plt.ylim(0,99)
plt.xlabel('Days', size=12)
plt.ylabel('PM2.5', size=12)
plt.legend(loc=1, prop={'size': 12})
plt.title("PM2.5 Activity - House 1")
plt.show()

In [ ]:
###############################################################################
################################## P M 2 . 5 ##################################
###############################################################################
################################ H O U S E   1 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["PM2.5"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["PM2.5"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["PM2.5"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=15, fontweight='bold')
plt.ylabel('PM2.5', size=15, fontweight='bold')
plt.ylim(top=13)
plt.legend(loc=1, prop={'size': 15})
plt.title("PM2.5 Calibration - House 1", size=18)
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["PM2.5"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["PM2.5"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["PM2.5"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('PM2.5', size=12)
plt.legend(loc=1, prop={'size': 15})
plt.title("PM2.5 Activity - House 1")
plt.show()

KR1_Cal_PM25_mean = Cal_KR["PM2.5"][Cal_KR["House"]==1].mean()
LR1_Cal_PM25_mean = Cal_LR["PM2.5"][Cal_LR["House"]==1].mean()
BR1_Cal_PM25_mean = Cal_BR["PM2.5"][Cal_BR["House"]==1].mean()

H1_Cal_PM25_mean = (KR1_Cal_PM25_mean + LR1_Cal_PM25_mean + BR1_Cal_PM25_mean)/3

KR1_PM25_adjustment = KR1_Cal_PM25_mean - H1_Cal_PM25_mean
LR1_PM25_adjustment = LR1_Cal_PM25_mean - H1_Cal_PM25_mean
BR1_PM25_adjustment = BR1_Cal_PM25_mean - H1_Cal_PM25_mean

IAQ_KR["adj_PM2.5"][IAQ_KR["House"]==1] = IAQ_KR["PM2.5"][IAQ_KR["House"]==1] - KR1_PM25_adjustment
IAQ_LR["adj_PM2.5"][IAQ_LR["House"]==1] = IAQ_LR["PM2.5"][IAQ_LR["House"]==1] - LR1_PM25_adjustment
IAQ_BR["adj_PM2.5"][IAQ_BR["House"]==1] = IAQ_BR["PM2.5"][IAQ_BR["House"]==1] - BR1_PM25_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_PM2.5"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["adj_PM2.5"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["adj_PM2.5"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=15, fontweight='bold')
plt.ylabel('PM2.5', size=15, fontweight='bold')
plt.legend(loc=1, prop={'size': 12})
plt.title("Adjusted PM2.5 Activity - House 1", size=18)
plt.show();

In [ ]:
###############################################################################
################################## P M 2 . 5 ##################################
###############################################################################
################################ H O U S E   2 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["PM2.5"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["PM2.5"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["PM2.5"][Cal_BR["House"] == 2], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('PM2.5', size=12)
plt.legend(loc=1, prop={'size': 12})
plt.title("PM2.5 Calibration - House 2")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["PM2.5"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["PM2.5"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["PM2.5"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('PM2.5', size=12)
plt.legend(loc=1, prop={'size': 12})
plt.title("PM2.5 Activity - House 2")
plt.show()

KR2_Cal_PM25_mean = Cal_KR["PM2.5"][Cal_KR["House"]==2].mean()
LR2_Cal_PM25_mean = Cal_LR["PM2.5"][Cal_LR["House"]==2].mean()
BR2_Cal_PM25_mean = Cal_BR["PM2.5"][Cal_BR["House"]==2].mean()

H2_Cal_PM25_mean = (KR2_Cal_PM25_mean + LR2_Cal_PM25_mean + BR2_Cal_PM25_mean)/3

KR2_PM25_adjustment = KR2_Cal_PM25_mean - H2_Cal_PM25_mean
LR2_PM25_adjustment = LR2_Cal_PM25_mean - H2_Cal_PM25_mean
BR2_PM25_adjustment = BR2_Cal_PM25_mean - H2_Cal_PM25_mean

IAQ_KR["adj_PM2.5"][IAQ_KR["House"]==2] = IAQ_KR["PM2.5"][IAQ_KR["House"]==2] - KR1_PM25_adjustment
IAQ_LR["adj_PM2.5"][IAQ_LR["House"]==2] = IAQ_LR["PM2.5"][IAQ_LR["House"]==2] - LR1_PM25_adjustment
IAQ_BR["adj_PM2.5"][IAQ_BR["House"]==2] = IAQ_BR["PM2.5"][IAQ_BR["House"]==2] - BR1_PM25_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_PM2.5"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["adj_PM2.5"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["adj_PM2.5"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=15, fontweight='bold')
plt.ylabel('PM2.5', size=15, fontweight='bold')
plt.legend(loc=1, prop={'size': 12})
plt.title("Adjusted PM2.5 Activity - House 2", size=18)
plt.show();

In [ ]:
###############################################################################
#################################### C O 2 ####################################
###############################################################################
################################ H O U S E   1 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["CO2"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["CO2"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["CO2"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('CO2', size=12)
plt.legend(loc=1, prop={'size': 12})
plt.title("CO2 Calibration - House 1")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["CO2"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["CO2"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["CO2"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('CO2', size=12)
plt.legend(loc=1, prop={'size': 12})
plt.title("CO2 Activity - House 1")
plt.show()

KR1_Cal_CO2_mean = Cal_KR["CO2"][Cal_KR["House"]==1].mean()
LR1_Cal_CO2_mean = Cal_LR["CO2"][Cal_LR["House"]==1].mean()
BR1_Cal_CO2_mean = Cal_BR["CO2"][Cal_BR["House"]==1].mean()

H1_Cal_CO2_mean = (KR1_Cal_CO2_mean + LR1_Cal_CO2_mean + BR1_Cal_CO2_mean)/3

KR1_CO2_adjustment = KR1_Cal_CO2_mean - H1_Cal_CO2_mean
LR1_CO2_adjustment = LR1_Cal_CO2_mean - H1_Cal_CO2_mean
BR1_CO2_adjustment = BR1_Cal_CO2_mean - H1_Cal_CO2_mean

IAQ_KR["adj_CO2"][IAQ_KR["House"]==1] = IAQ_KR["CO2"][IAQ_KR["House"]==1] - KR1_CO2_adjustment
IAQ_LR["adj_CO2"][IAQ_LR["House"]==1] = IAQ_LR["CO2"][IAQ_LR["House"]==1] - LR1_CO2_adjustment
IAQ_BR["adj_CO2"][IAQ_BR["House"]==1] = IAQ_BR["CO2"][IAQ_BR["House"]==1] - BR1_CO2_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_CO2"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["adj_CO2"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["adj_CO2"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('CO2', size=12)
plt.legend(loc=1, prop={'size': 12})
plt.title("Adjusted CO2 Activity - House 1")
plt.show();

In [ ]:
###############################################################################
#################################### C O 2 ####################################
###############################################################################
################################ H O U S E   2 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["CO2"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["CO2"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["CO2"][Cal_BR["House"] == 2], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('CO2', size=12)
plt.legend(prop={'size': 12})
plt.title("CO2 Calibration - House 2")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["CO2"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["CO2"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["CO2"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('CO2', size=12)
plt.legend(prop={'size': 12})
plt.title("CO2 Activity - House 2")
plt.show()

KR2_Cal_CO2_mean = Cal_KR["CO2"][Cal_KR["House"]==2].mean()
LR2_Cal_CO2_mean = Cal_LR["CO2"][Cal_LR["House"]==2].mean()
BR2_Cal_CO2_mean = Cal_BR["CO2"][Cal_BR["House"]==2].mean()

H2_Cal_CO2_mean = (KR2_Cal_CO2_mean + LR2_Cal_CO2_mean + BR2_Cal_CO2_mean)/3

KR2_CO2_adjustment = KR2_Cal_CO2_mean - H2_Cal_CO2_mean
LR2_CO2_adjustment = LR2_Cal_CO2_mean - H2_Cal_CO2_mean
BR2_CO2_adjustment = BR2_Cal_CO2_mean - H2_Cal_CO2_mean

IAQ_KR["adj_CO2"][IAQ_KR["House"]==2] = IAQ_KR["CO2"][IAQ_KR["House"]==2] - KR1_CO2_adjustment
IAQ_LR["adj_CO2"][IAQ_LR["House"]==2] = IAQ_LR["CO2"][IAQ_LR["House"]==2] - LR1_CO2_adjustment
IAQ_BR["adj_CO2"][IAQ_BR["House"]==2] = IAQ_BR["CO2"][IAQ_BR["House"]==2] - BR1_CO2_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_CO2"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["adj_CO2"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["adj_CO2"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('CO2', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted CO2 Activity - House 2")
plt.show();

In [ ]:
###############################################################################
################################### T V O C ###################################
###############################################################################
################################ H O U S E   1 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["TVOC"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["TVOC"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["TVOC"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("TVOC Calibration - House 1")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["TVOC"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["TVOC"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["TVOC"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("TVOC Activity - House 1")
plt.show()


KR1_Cal_TVOC_mean = Cal_KR["TVOC"][Cal_KR["House"]==1].mean()
LR1_Cal_TVOC_mean = Cal_LR["TVOC"][Cal_LR["House"]==1].mean()
BR1_Cal_TVOC_mean = Cal_BR["TVOC"][Cal_BR["House"]==1].mean()

H1_Cal_TVOC_mean = (KR1_Cal_TVOC_mean + BR1_Cal_TVOC_mean)/2

KR1_TVOC_adjustment = KR1_Cal_TVOC_mean - H1_Cal_TVOC_mean
LR1_TVOC_adjustment = LR1_Cal_TVOC_mean - H1_Cal_TVOC_mean
BR1_TVOC_adjustment = BR1_Cal_TVOC_mean - H1_Cal_TVOC_mean

IAQ_KR["adj_TVOC"] = IAQ_KR["TVOC"][IAQ_KR["House"]==1] - KR1_TVOC_adjustment
IAQ_LR["adj_TVOC"] = IAQ_LR["TVOC"][IAQ_LR["House"]==1] - LR1_TVOC_adjustment
IAQ_BR["adj_TVOC"] = IAQ_BR["TVOC"][IAQ_BR["House"]==1] - BR1_TVOC_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_TVOC"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["adj_TVOC"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["adj_TVOC"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted TVOC Activity - House 1")
plt.show();

In [ ]:
###############################################################################
################################### T V O C ###################################
###############################################################################
################################ H O U S E   2 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["TVOC"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["TVOC"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["TVOC"][Cal_BR["House"] == 2], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("TVOC Calibration - House 2")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["TVOC"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["TVOC"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["TVOC"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("TVOC Activity - House 2")
plt.show()

KR2_Cal_TVOC_mean = Cal_KR["TVOC"][Cal_KR["House"]==2].mean()
LR2_Cal_TVOC_mean = Cal_LR["TVOC"][Cal_LR["House"]==2].mean()
BR2_Cal_TVOC_mean = Cal_BR["TVOC"][Cal_BR["House"]==2].mean()

H2_Cal_TVOC_mean = (KR2_Cal_TVOC_mean + LR2_Cal_TVOC_mean + BR2_Cal_TVOC_mean)/3

KR2_TVOC_adjustment = KR2_Cal_TVOC_mean - H2_Cal_TVOC_mean
LR2_TVOC_adjustment = LR2_Cal_TVOC_mean - H2_Cal_TVOC_mean
BR2_TVOC_adjustment = BR2_Cal_TVOC_mean - H2_Cal_TVOC_mean

IAQ_KR["adj_TVOC"][IAQ_KR["House"]==2] = IAQ_KR["TVOC"][IAQ_KR["House"]==2] - KR2_TVOC_adjustment
IAQ_LR["adj_TVOC"][IAQ_LR["House"]==2] = IAQ_LR["TVOC"][IAQ_LR["House"]==2] - LR2_TVOC_adjustment
IAQ_BR["adj_TVOC"][IAQ_BR["House"]==2] = IAQ_BR["TVOC"][IAQ_BR["House"]==2] - BR2_TVOC_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_TVOC"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["adj_TVOC"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["adj_TVOC"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted TVOC Activity - House 2")
plt.show();

Due to how big the discrepancy is between the Living Room sensor and all the others during House 2's calibration period, applying the same formula we've been using before puts this value into the negative spectrum. As both houses have an open plan Kitchen and the kitchen and living room sensors show a high level of accordance, we'll use the calibration data from the Kitchen for the living room sensor, pending a better idea.

In [ ]:
###############################################################################
####################### T V O C - A L T E R N A T I V E #######################
###############################################################################
################################ H O U S E   2 ################################
###############################################################################

#This is the line we've modified
LR2_TVOC_adjustment = KR2_Cal_TVOC_mean - H2_Cal_TVOC_mean

IAQ_LR["adj_TVOC"][IAQ_LR["House"]==2] = IAQ_LR["TVOC"][IAQ_LR["House"]==2] - LR2_TVOC_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_TVOC"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["adj_TVOC"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["adj_TVOC"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted TVOC Activity - House 2")
plt.show();


#We did the same for House 1 as like House 2, House 1's Kitchen is connected to the Living Room with only a half-way in between
LR1_TVOC_adjustment = KR1_Cal_TVOC_mean - H1_Cal_TVOC_mean

IAQ_LR["adj_TVOC"][IAQ_LR["House"]==1] = IAQ_LR["TVOC"][IAQ_LR["House"]==1] - LR1_TVOC_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_TVOC"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["adj_TVOC"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["adj_TVOC"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('TVOC (Total Volatile Organic Compounds)', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted TVOC Activity - House 1")
plt.show();

In [ ]:
###############################################################################
############################ T E M P E R A T U R E ############################
###############################################################################
################################ H O U S E   1 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["Temp"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["Temp"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["Temp"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('Temperature', size=12)
plt.legend(prop={'size': 12})
plt.title("Temp Calibration - House 1")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["Temp"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["Temp"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["Temp"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Temperature', size=12)
plt.legend(prop={'size': 12})
plt.title("Temp Activity - House 1")
plt.show()

KR1_Cal_Temp_mean = Cal_KR["Temp"][Cal_KR["House"]==1].mean()
LR1_Cal_Temp_mean = Cal_LR["Temp"][Cal_LR["House"]==1].mean()
BR1_Cal_Temp_mean = Cal_BR["Temp"][Cal_BR["House"]==1].mean()

H1_Cal_Temp_mean = (KR1_Cal_Temp_mean + LR1_Cal_Temp_mean + BR1_Cal_Temp_mean)/3

KR1_Temp_adjustment = KR1_Cal_Temp_mean - H1_Cal_Temp_mean
LR1_Temp_adjustment = LR1_Cal_Temp_mean - H1_Cal_Temp_mean
BR1_Temp_adjustment = BR1_Cal_Temp_mean - H1_Cal_Temp_mean

IAQ_KR["adj_Temp"][IAQ_KR["House"]==1] = IAQ_KR["Temp"][IAQ_KR["House"]==1] - KR1_Temp_adjustment
IAQ_LR["adj_Temp"][IAQ_LR["House"]==1] = IAQ_LR["Temp"][IAQ_LR["House"]==1] - LR1_Temp_adjustment
IAQ_BR["adj_Temp"][IAQ_BR["House"]==1] = IAQ_BR["Temp"][IAQ_BR["House"]==1] - BR1_Temp_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_Temp"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["adj_Temp"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["adj_Temp"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Temperature', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted Temp Activity - House 1")
plt.show();



In [ ]:
###############################################################################
############################ T E M P E R A T U R E ############################
###############################################################################
################################ H O U S E   2 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["Temp"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["Temp"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["Temp"][Cal_BR["House"] == 2], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('Temperature', size=12)
plt.legend(prop={'size': 12})
plt.title("Temp Calibration - House 2")
plt.show();

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["Temp"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["Temp"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["Temp"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Temperature', size=12)
plt.legend(prop={'size': 12})
plt.title("Temp Activity - House 2")
plt.show()

KR2_Cal_Temp_mean = Cal_KR["Temp"][Cal_KR["House"]==2].mean()
LR2_Cal_Temp_mean = Cal_LR["Temp"][Cal_LR["House"]==2].mean()
BR2_Cal_Temp_mean = Cal_BR["Temp"][Cal_BR["House"]==2].mean()

H2_Cal_Temp_mean = (KR2_Cal_Temp_mean + LR2_Cal_Temp_mean + BR2_Cal_Temp_mean)/3

KR2_Temp_adjustment = KR2_Cal_Temp_mean - H2_Cal_Temp_mean
LR2_Temp_adjustment = LR2_Cal_Temp_mean - H2_Cal_Temp_mean
BR2_Temp_adjustment = BR2_Cal_Temp_mean - H2_Cal_Temp_mean

IAQ_KR["adj_Temp"][IAQ_KR["House"]==2] = IAQ_KR["Temp"][IAQ_KR["House"]==2] - KR1_Temp_adjustment
IAQ_LR["adj_Temp"][IAQ_LR["House"]==2] = IAQ_LR["Temp"][IAQ_LR["House"]==2] - LR1_Temp_adjustment
IAQ_BR["adj_Temp"][IAQ_BR["House"]==2] = IAQ_BR["Temp"][IAQ_BR["House"]==2] - BR1_Temp_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_Temp"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["adj_Temp"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["adj_Temp"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Temperature', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted Temp Activity - House 2")
plt.show();

In [ ]:
###############################################################################
###################### R E L A T I V E   H U M I D I T Y ######################
###############################################################################
################################ H O U S E   1 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["RH"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["RH"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["RH"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('Relative Humidity', size=12)
plt.legend(prop={'size': 12})
plt.title("Relative Humidity Calibration - House 1")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["RH"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["RH"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["RH"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Relative Humidity', size=12)
plt.legend(prop={'size': 12})
plt.title("Relative Humidity Activity - House 1")
plt.show()

KR1_Cal_RH_mean = Cal_KR["RH"][Cal_KR["House"]==1].mean()
LR1_Cal_RH_mean = Cal_LR["RH"][Cal_LR["House"]==1].mean()
BR1_Cal_RH_mean = Cal_BR["RH"][Cal_BR["House"]==1].mean()

H1_Cal_RH_mean = (KR1_Cal_RH_mean + LR1_Cal_RH_mean + BR1_Cal_RH_mean)/3

KR1_RH_adjustment = KR1_Cal_RH_mean - H1_Cal_RH_mean
LR1_RH_adjustment = LR1_Cal_RH_mean - H1_Cal_RH_mean
BR1_RH_adjustment = BR1_Cal_RH_mean - H1_Cal_RH_mean

IAQ_KR["adj_RH"][IAQ_KR["House"]==1] = IAQ_KR["RH"][IAQ_KR["House"]==1] - KR1_RH_adjustment
IAQ_LR["adj_RH"][IAQ_LR["House"]==1] = IAQ_LR["RH"][IAQ_LR["House"]==1] - LR1_RH_adjustment
IAQ_BR["adj_RH"][IAQ_BR["House"]==1] = IAQ_BR["RH"][IAQ_BR["House"]==1] - BR1_RH_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_RH"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["adj_RH"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["adj_RH"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Relative Humidity', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted Relative Humidity Activity - House 1")
plt.show();

In [ ]:
###############################################################################
###################### R E L A T I V E   H U M I D I T Y ######################
###############################################################################
################################ H O U S E   2 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["RH"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["RH"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["RH"][Cal_BR["House"] == 2], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('Relative Humidity', size=12)
plt.legend(prop={'size': 12})
plt.title("Relative Humidity Calibration - House 2")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["RH"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["RH"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["RH"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Relative Humidity', size=12)
plt.legend(prop={'size': 12})
plt.title("Relative Humidity Activity - House 2")
plt.show()

KR2_Cal_RH_mean = Cal_KR["RH"][Cal_KR["House"]==2].mean()
LR2_Cal_RH_mean = Cal_LR["RH"][Cal_LR["House"]==2].mean()
BR2_Cal_RH_mean = Cal_BR["RH"][Cal_BR["House"]==2].mean()

H2_Cal_RH_mean = (KR2_Cal_RH_mean + LR2_Cal_RH_mean + BR2_Cal_RH_mean)/3

KR2_RH_adjustment = KR2_Cal_RH_mean - H2_Cal_RH_mean
LR2_RH_adjustment = LR2_Cal_RH_mean - H2_Cal_RH_mean
BR2_RH_adjustment = BR2_Cal_RH_mean - H2_Cal_RH_mean

IAQ_KR["adj_RH"][IAQ_KR["House"]==2] = IAQ_KR["RH"][IAQ_KR["House"]==2] - KR2_RH_adjustment

#For the Living Room adjustement, we know that the house is very small and both Kitchen and Living Room are joined together
# We notice that the calibration gap between the Kitchen and Living room sensors is around 5, however in the actual data
# the spread appears to be in the order of 10 units. As such, to our LR2 adjustement, we add the difference we saw, 5
IAQ_LR["adj_RH"][IAQ_LR["House"]==2] = IAQ_LR["RH"][IAQ_LR["House"]==2] - LR2_RH_adjustment+5
IAQ_BR["adj_RH"][IAQ_BR["House"]==2] = IAQ_BR["RH"][IAQ_BR["House"]==2] - BR2_RH_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_RH"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["adj_RH"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["adj_RH"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Relative Humidity', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted Relative Humidity Activity - House 2")
plt.show();


In [ ]:
###############################################################################
########################### A I R   P R E S S U R E ###########################
###############################################################################
################################ H O U S E   1 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["AirPres"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["AirPres"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["AirPres"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('Air Pressure', size=12)
plt.legend(prop={'size': 12})
plt.title("AirPres Calibration - House 1")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["AirPres"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["AirPres"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["AirPres"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Air Pressure', size=12)
plt.legend(prop={'size': 12})
plt.title("AirPres Activity - House 1")
plt.show()

KR1_Cal_AirPres_mean = Cal_KR["AirPres"][Cal_KR["House"]==1].mean()
LR1_Cal_AirPres_mean = Cal_LR["AirPres"][Cal_LR["House"]==1].mean()
BR1_Cal_AirPres_mean = Cal_BR["AirPres"][Cal_BR["House"]==1].mean()

H1_Cal_AirPres_mean = (KR1_Cal_AirPres_mean + LR1_Cal_AirPres_mean + BR1_Cal_AirPres_mean)/3

KR1_AirPres_adjustment = KR1_Cal_AirPres_mean - H1_Cal_AirPres_mean
LR1_AirPres_adjustment = LR1_Cal_AirPres_mean - H1_Cal_AirPres_mean
BR1_AirPres_adjustment = BR1_Cal_AirPres_mean - H1_Cal_AirPres_mean

IAQ_KR["adj_AirPres"][IAQ_KR["House"]==1] = IAQ_KR["AirPres"][IAQ_KR["House"]==1] - KR1_AirPres_adjustment
IAQ_LR["adj_AirPres"][IAQ_LR["House"]==1] = IAQ_LR["AirPres"][IAQ_LR["House"]==1] - LR1_AirPres_adjustment
IAQ_BR["adj_AirPres"][IAQ_BR["House"]==1] = IAQ_BR["AirPres"][IAQ_BR["House"]==1] - BR1_AirPres_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_AirPres"][IAQ_KR["House"] == 1], label='Kitchen')
plt.plot(IAQ_LR["adj_AirPres"][IAQ_LR["House"] == 1], label='Living Room')
plt.plot(IAQ_BR["adj_AirPres"][IAQ_BR["House"] == 1], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Air Pressure', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted AirPres Activity - House 1")
plt.show();

In [ ]:
###############################################################################
########################### A I R   P R E S S U R E ###########################
###############################################################################
################################ H O U S E   2 ################################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["AirPres"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["AirPres"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["AirPres"][Cal_BR["House"] == 2], label='Bedroom')
plt.xlabel('Hours', size=12)
plt.ylabel('Air Pressure', size=12)
plt.legend(prop={'size': 12})
plt.title("AirPres Calibration - House 2")
plt.show()

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["AirPres"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["AirPres"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["AirPres"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Air Pressure', size=12)
plt.legend(prop={'size': 12})
plt.title("AirPres Activity - House 2")
plt.show()

KR2_Cal_AirPres_mean = Cal_KR["AirPres"][Cal_KR["House"]==2].mean()
LR2_Cal_AirPres_mean = Cal_LR["AirPres"][Cal_LR["House"]==2].mean()
BR2_Cal_AirPres_mean = Cal_BR["AirPres"][Cal_BR["House"]==2].mean()

H2_Cal_AirPres_mean = (KR2_Cal_AirPres_mean + LR2_Cal_AirPres_mean + BR2_Cal_AirPres_mean)/3

KR2_AirPres_adjustment = KR2_Cal_AirPres_mean - H2_Cal_AirPres_mean
LR2_AirPres_adjustment = LR2_Cal_AirPres_mean - H2_Cal_AirPres_mean
BR2_AirPres_adjustment = BR2_Cal_AirPres_mean - H2_Cal_AirPres_mean

IAQ_KR["adj_AirPres"][IAQ_KR["House"]==2] = IAQ_KR["AirPres"][IAQ_KR["House"]==2] - KR2_AirPres_adjustment
IAQ_LR["adj_AirPres"][IAQ_LR["House"]==2] = IAQ_LR["AirPres"][IAQ_LR["House"]==2] - LR2_AirPres_adjustment
IAQ_BR["adj_AirPres"][IAQ_BR["House"]==2] = IAQ_BR["AirPres"][IAQ_BR["House"]==2] - BR2_AirPres_adjustment

plt.figure(figsize=(15,5))
plt.plot(IAQ_KR["adj_AirPres"][IAQ_KR["House"] == 2], label='Kitchen')
plt.plot(IAQ_LR["adj_AirPres"][IAQ_LR["House"] == 2], label='Living Room')
plt.plot(IAQ_BR["adj_AirPres"][IAQ_BR["House"] == 2], label='Bedroom')
plt.xlabel('Days', size=12)
plt.ylabel('Air Pressure', size=12)
plt.legend(prop={'size': 12})
plt.title("Adjusted AirPres Activity - House 2")
plt.show();

In [ ]:
###############################################################################
############################ O Z O N E   &   N O 2 ############################
###############################################################################

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["Ozone"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["Ozone"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["Ozone"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=15, fontweight='bold')
plt.ylabel('Ozone', size=15, fontweight='bold')
plt.legend(loc=1, prop={'size': 12})
plt.title("Ozone Calibration - House 1", size=18)
plt.show()

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["Ozone"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["Ozone"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["Ozone"][Cal_BR["House"] == 2], label='Bedroom')
plt.legend(loc=1, prop={'size': 12})
plt.xlabel('Hours', size=15, fontweight='bold')
plt.ylabel('Ozone', size=15, fontweight='bold')
plt.title("Ozone Calibration - House 2", size=18)
plt.show();

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["NO2"][Cal_KR["House"] == 1], label='Kitchen')
plt.plot(Cal_LR["NO2"][Cal_LR["House"] == 1], label='Living Room')
plt.plot(Cal_BR["NO2"][Cal_BR["House"] == 1], label='Bedroom')
plt.xlabel('Hours', size=15, fontweight='bold')
plt.ylabel('NO2', size=15, fontweight='bold')
plt.legend(loc=1, prop={'size': 12})
plt.title("NO2 Calibration - House 1", size=18)
plt.show()

plt.figure(figsize=(15,5))
plt.plot(Cal_KR["NO2"][Cal_KR["House"] == 2], label='Kitchen')
plt.plot(Cal_LR["NO2"][Cal_LR["House"] == 2], label='Living Room')
plt.plot(Cal_BR["NO2"][Cal_BR["House"] == 2], label='Bedroom')
plt.xlabel('Hours', size=15, fontweight='bold')
plt.ylabel('NO2', size=15, fontweight='bold')
plt.legend(loc=1, prop={'size': 12})
plt.title("NO2 Calibration - House 2", size=18)
plt.show();

In [ ]:
#We're excluding Ozone and NO2 from our model since we couldn't find any agreement in the calibration data
#Additionally, we're removing all our original features and renaming the adjusted ones to what the original were called

IAQ_LR.drop(["NO2", "Ozone", "Temp", "RH", "PM2.5", "TVOC", "CO2", "AirPres"], axis=1, inplace=True)
IAQ_KR.drop(["NO2", "Ozone", "Temp", "RH", "PM2.5", "TVOC", "CO2", "AirPres"], axis=1, inplace=True)
IAQ_BR.drop(["NO2", "Ozone", "Temp", "RH", "PM2.5", "TVOC", "CO2", "AirPres"], axis=1, inplace=True)

IAQ_LR.columns = IAQ_LR.columns.str.replace("adj_", "")
IAQ_KR.columns = IAQ_KR.columns.str.replace("adj_", "")
IAQ_BR.columns = IAQ_BR.columns.str.replace("adj_", "")

#IAQ_LR.to_csv(r'treated datasets/IAQ_LR.csv')
#IAQ_KR.to_csv(r'treated datasets/IAQ_KR.csv')
#IAQ_BR.to_csv(r'treated datasets/IAQ_BR.csv')

#Here we join the sensor data from our 3 rooms, Living Room, Kitchen and Bedroom with all activity available

LR = pd.merge(IAQ_LR, activity, left_index=True, right_index=True)
KR = pd.merge(IAQ_KR, activity, left_index=True, right_index=True)
BR = pd.merge(IAQ_BR, activity, left_index=True, right_index=True)



small_LR = pd.merge(IAQ_LR, activity_LR, left_index=True, right_index=True)
small_KR = pd.merge(IAQ_KR, activity_KR, left_index=True, right_index=True)
small_BR = pd.merge(IAQ_BR, activity_BR, left_index=True, right_index=True)


#LR.to_csv(r'treated datasets/LivingRoom.csv')
#KR.to_csv(r'treated datasets/Kitchen.csv')
#BR.to_csv(r'treated datasets/Bedroom.csv')

In [ ]:
#In case having +30 variables for the individual rooms turns out to be too much and make our models unstable, we'll model with
#only the variables for the room in question

small_LR = LR.copy()
for i in small_LR.columns:
    if '_KR' in i:
        small_LR = small_LR.drop([i], axis=1)
    elif '_BR' in i:
        small_LR = small_LR.drop([i], axis=1)

        
small_KR = KR.copy()
for i in small_KR.columns:
    if '_LR' in i:
        small_KR = small_KR.drop([i], axis=1)
    elif '_BR' in i:
        small_KR = small_KR.drop([i], axis=1)


small_BR = BR.copy()
for i in small_BR.columns:
    if '_LR' in i:
        small_BR = small_BR.drop([i], axis=1)
    elif '_KR' in i:
        small_BR = small_BR.drop([i], axis=1)
        

small_LR.columns = small_LR.columns.str.replace("_LR", "")
small_KR.columns = small_KR.columns.str.replace("_KR", "")
small_BR.columns = small_BR.columns.str.replace("_BR", "")

In [ ]:
#Here we create datasets for each room in each house and one for both rooms together. We do this as House 2 has slightly more
#features available than House 1. The purpose of this split is to analyze which features are most impactful to PM2.5 and if
#these features are equally impactful in both houses or not.

LR1 = LR[LR["House"] == 1].dropna(axis=1)
LR2 = LR[LR["House"] == 2].dropna(axis=1)
LR  = LR.dropna(axis=1)

KR1 = KR[KR["House"] == 1].dropna(axis=1)
KR2 = KR[KR["House"] == 2].dropna(axis=1)
KR  = KR.dropna(axis=1)

BR1 = BR[BR["House"] == 1].dropna(axis=1)
BR2 = BR[BR["House"] == 2].dropna(axis=1)
BR  = BR.dropna(axis=1)


small_LR1 = small_LR[small_LR["House"] == 1].dropna(axis=1)
small_LR2 = small_LR[small_LR["House"] == 2].dropna(axis=1)
small_LR  = small_LR.dropna(axis=1)

small_KR1 = small_KR[small_KR["House"] == 1].dropna(axis=1)
small_KR2 = small_KR[small_KR["House"] == 2].dropna(axis=1)
small_KR  = small_KR.dropna(axis=1)

small_BR1 = small_BR[small_BR["House"] == 1].dropna(axis=1)
small_BR2 = small_BR[small_BR["House"] == 2].dropna(axis=1)
small_BR  = small_BR.dropna(axis=1)


#Creating a DF for House 1 (H1), House 2 (H2) and both houses (H) with rooms one-hot encoded
df1_1 = LR1.copy()
df1_2 = KR1.copy()
df1_3 = BR1.copy()

df2_1 = LR2.copy()
df2_2 = KR2.copy()
df2_3 = BR2.copy()

df1_1["LivingRoom"] = 1
df1_1["Kitchen"] = 0
df1_1["Bedroom"] = 0

df1_2["LivingRoom"] = 0
df1_2["Kitchen"] = 1
df1_2["Bedroom"] = 0

df1_3["LivingRoom"] = 0
df1_3["Kitchen"] = 0
df1_3["Bedroom"] = 1

df2_1["LivingRoom"] = 1
df2_1["Kitchen"] = 0
df2_1["Bedroom"] = 0

df2_2["LivingRoom"] = 0
df2_2["Kitchen"] = 1
df2_2["Bedroom"] = 0

df2_3["LivingRoom"] = 0
df2_3["Kitchen"] = 0
df2_3["Bedroom"] = 1


H1 = pd.concat([df1_1, df1_2, df1_3], axis=0, sort=False)

H2 = pd.concat([df2_1, df2_2, df2_3], axis=0, sort=False)
H2.Door_LR = H2.Door_LR.astype(int)
H2.Door_KR = H2.Door_KR.astype(int)
H2.Door_BR = H2.Door_BR.astype(int)
H2.Shower = H2.Shower.astype(int)

H = pd.concat([H1, H2], axis=0, sort=False).dropna(axis=1)

print("H shape:",H.shape)
print("H1 shape:",H1.shape)
print("H2 shape:",H2.shape)
print("\n")
print('-------------------------')
print("\n")
print("LR shape:",LR.shape)
print("LR1 shape:",LR1.shape)
print("LR2 shape,",LR2.shape)
print("\n")
print("KR shape:",KR.shape)
print("KR1 shape:",KR1.shape)
print("KR2 shape,",KR2.shape)
print("\n")
print("BR shape:",BR.shape)
print("BR1 shape:",BR1.shape)
print("BR2 shape,",BR2.shape)
print("\n")
print('-------------------------')
print("\n")
print("small_LR shape:",small_LR.shape)
print("small_LR1 shape:",small_LR1.shape)
print("small_LR2 shape,",small_LR2.shape)
print("\n")
print("small_KR shape:",small_KR.shape)
print("small_KR1 shape:",small_KR1.shape)
print("small_KR2 shape,",small_KR2.shape)
print("\n")
print("small_BR shape:",small_BR.shape)
print("small_BR1 shape:",small_BR1.shape)
print("small_BR2 shape,",small_BR2.shape)

In [ ]:
print (H1.columns)
print (H2.columns)
print("\n")
print("Features in House 2 unavailable in House 1: Door and Shower")

# Exploratory Data Analysis

## Goals of this section:

We're dealing with two similar layout houses. Both houses have the Kitchen joined with the Living room and then a bedroom too.

1 - Comparing means, standard deviations, mins and maximums between rooms in House 1 and 2; 6 comparisons.  
2 - Comparing means between Activities' 1s and 0s. A "1" represents "On/Open/Yes", such as Heating On, Door Open, Frying occurred.  
3 - We'll look into collinearity in this section as well as to prevent having a model with many features irrelevant to the change of PM2.5  
4 - Related to the mins/max comparison, we'll look at outliers and consult our activity logs to try and explain what could have happened there

In [ ]:
display(H1.describe())
display(H2.describe())

In [ ]:
LR1_mean = round(LR1["PM2.5"].mean(),4)
LR1_std = round(LR1["PM2.5"].std(),4)
LR1_min = round(LR1["PM2.5"].min(),4)
LR1_max = round(LR1["PM2.5"].max(),4)

KR1_mean = round(KR1["PM2.5"].mean(),4)
KR1_std = round(KR1["PM2.5"].std(),4)
KR1_min = round(KR1["PM2.5"].min(),4)
KR1_max = round(KR1["PM2.5"].max(),4)

BR1_mean = round(BR1["PM2.5"].mean(),4)
BR1_std = round(BR1["PM2.5"].std(),4)
BR1_min = round(BR1["PM2.5"].min(),4)
BR1_max = round(BR1["PM2.5"].max(),4)

LR2_mean = round(LR2["PM2.5"].mean(),4)
LR2_std = round(LR2["PM2.5"].std(),4)
LR2_min = round(LR2["PM2.5"].min(),4)
LR2_max = round(LR2["PM2.5"].max(),4)

KR2_mean = round(KR2["PM2.5"].mean(),4)
KR2_std = round(KR2["PM2.5"].std(),4)
KR2_min = round(KR2["PM2.5"].min(),4)
KR2_max = round(KR2["PM2.5"].max(),4)

BR2_mean = round(BR2["PM2.5"].mean(),4)
BR2_std = round(BR2["PM2.5"].std(),4)
BR2_min = round(BR2["PM2.5"].min(),4)
BR2_max = round(BR2["PM2.5"].max(),4)

H1_mean = round(H1["PM2.5"].mean(),4)
H1_std = round(H1["PM2.5"].std(),4)
H1_min = round(H1["PM2.5"].min(),4)
H1_max = round(H1["PM2.5"].max(),4)

H2_mean = round(H2["PM2.5"].mean(),4)
H2_std = round(H2["PM2.5"].std(),4)
H2_min = round(H2["PM2.5"].min(),4)
H2_max = round(H2["PM2.5"].max(),4)


descriptives = {'Mean':[LR1_mean, KR1_mean, BR1_mean, LR2_mean, KR2_mean, BR2_mean, H1_mean, H2_mean], 
               'STD':[LR1_std, KR1_std, BR1_std, LR2_std, KR2_std, BR2_std, H1_std, H2_std],
               'Min':[LR1_min, KR1_min, BR1_min, LR2_min, KR2_min, BR2_min, H1_min, H2_min],
               'Max':[LR1_max, KR1_max, BR1_max, LR2_max, KR2_max, BR2_max, H1_max, H2_max],} 

descriptive_df = pd.DataFrame(descriptives, index =['LR1', 'KR1', 'BR1', 'LR2', 'KR2', 'BR2', 'H1', 'H2'])


- LR1 and KR1 are very similar as they refer to a shared Kitchen and Living room. The sensors have been placed on opposite sides of the room

- BR seems to have the smallest deviation in data from both sets which is to be expected as activities that generate PM2.5 (cooking, fireplace) aren't common there

- Like LR1 and KR1, KR2 and LR2 refer to a joint Kitchen and Living room, however there's a clear increase in the mean on LR2. We'll have to investigate that further.

### Here we look for the most correlated features per room in each house. Count of activity occurrences and causation not accounted for here, we're just looking at the big picture for now and see if we have similar features affecting the same houses and/or the same rooms.

In [ ]:
roompm25 = {'House 1':[LR1_mean, KR1_mean, BR1_mean], 'House 2':[LR2_mean, KR2_mean, BR2_mean]} 
roompm25_df = pd.DataFrame(roompm25, index =['Living Room', 'Kitchen', 'Bedroom']).T

ax = roompm25_df[['Kitchen', 'Living Room', 'Bedroom']].plot(kind='bar',
              figsize=(15,5), color=['royalblue', 'darkorange', 'forestgreen'], edgecolor='black',fontsize=13);
ax.set_alpha(0.8)
ax.legend(loc = 9, prop={'size': 15})
ax.set_title('Mean PM2.5 by Room', fontsize=18)
#ax.set_title("Overall PM2.5 mean per Room",
#fontsize=18)
ax.set_ylim([0, 23])
ax.set_ylabel('PM2.5', size=15, fontweight='bold')
ax.set_xticklabels(labels=['House 1', 'House 2'], size=15, rotation=0, fontweight='bold')
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x()+0.04, i.get_height()+0.4, \
            str(round((i.get_height()), 2)), fontsize=15, color='dimgrey', rotation=0, fontweight='bold')

In [ ]:

ax.set_alpha(0.8)

ax.set_ylabel("RMSE", fontsize=15, fontweight='bold');
ax.set_xticklabels(score_df.index,rotation=0, fontsize=15, fontweight='bold')
ax.set_ylim([0, 15])

for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x()+0.015, i.get_height()+0.1, \
            str(round((i.get_height()), 2)), fontsize=12, color='black',
                rotation=0, fontweight='bold')

### With the distribution plots for House 1, House 2 and their combination, we aim to look at how PM2.5 is distributed and where the possible outliers lie in.

In [ ]:
#Distribution plots for House 1 and 2

display(descriptive_df.loc[['H1', 'H2']])

plt.figure(figsize=(15,5))
sns.distplot(H1["PM2.5"])
plt.xlim(0, 150)
plt.ylim(0, 0.12)
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.title("PM2.5 - House 1", size=20)
plt.show()

plt.figure(figsize=(15,5))
sns.distplot(H2["PM2.5"])
plt.xlim(0, 150)
plt.ylim(0, 0.12)
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.title("PM2.5 - House 2", size=20)
plt.show()

################################## Superimposed distplot ##################################

plt.figure(figsize=(15,5))
sns.distplot(H1["PM2.5"], label = 'House 1')
sns.distplot(H2["PM2.5"], label = 'House 2')
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.xlim(0, 150)
plt.ylim(0, 0.12)
plt.title("PM2.5 - Houses", size=20)
plt.legend(loc=1, prop={'size': 12})
plt.show()

In [ ]:
#Distribution plots for House 1

display(descriptive_df.loc[['LR1', 'KR1', 'BR1']])

plt.figure(figsize=(15,5))
sns.distplot(LR1["PM2.5"])
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.xlim(0, 80)
plt.ylim(0, 0.12)
plt.title("PM2.5 - House 1 - Living Room", size=20)
plt.show()

plt.figure(figsize=(15,5))
sns.distplot(KR1["PM2.5"])
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.xlim(0, 80)
plt.ylim(0, 0.12)
plt.title("PM2.5 - House 1 - Kitchen", size=20)
plt.show()

plt.figure(figsize=(15,5))
sns.distplot(BR1["PM2.5"])
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.xlim(0, 80)
plt.ylim(0, 0.12)
plt.title("PM2.5 - House 1 - Bedroom", size=20)
plt.show()


################################## Superimposed distplot ##################################

plt.figure(figsize=(15,5))
sns.distplot(LR1["PM2.5"], label = 'Living Room')
sns.distplot(KR1["PM2.5"], label = 'Kitchen')
sns.distplot(BR1["PM2.5"], label = 'Bedroom')
plt.xticks(size=15)
plt.xlabel('PM2.5', size=15, fontweight='bold')
plt.yticks(size=10)
plt.xlim(0, 80)
plt.ylim(0, 0.12)
plt.ylabel('Density', size=15, fontweight='bold')
plt.title("PM2.5 - House 1", size=18)
plt.legend(loc=1, prop={'size': 15})
plt.show()

We immediately see the Kitchen and Living room showing some outliers on the 40 μg/m3 mark. This will need to be looked into.

In [ ]:
#Distribution plots for House 2

display(descriptive_df.loc[['LR2', 'KR2', 'BR2']])

plt.figure(figsize=(15,5))
sns.distplot(LR2["PM2.5"])
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.xlim(0, 150)
plt.ylim(0, 0.1)
plt.title("PM2.5 - House 2 - Living Room", size=20)
plt.show()

plt.figure(figsize=(15,5))
sns.distplot(KR2["PM2.5"])
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.xlim(0, 150)
plt.ylim(0, 0.1)
plt.title("PM2.5 - House 2 - Kitchen", size=20)
plt.show()

plt.figure(figsize=(15,5))
sns.distplot(BR2["PM2.5"])
plt.xticks(size=15)
plt.xlabel('')
plt.yticks(size=12)
plt.xlim(0, 150)
plt.ylim(0, 0.1)
plt.title("PM2.5 - House 2 - Bedroom", size=20)
plt.show();


################################## Superimposed distplot ##################################

plt.figure(figsize=(15,5))
sns.distplot(LR2["PM2.5"], label = 'Living Room')
sns.distplot(KR2["PM2.5"], label = 'Kitchen')
sns.distplot(BR2["PM2.5"], label = 'Bedroom')
plt.xticks(size=15)
plt.xlabel('PM2.5', size=15, fontweight='bold')
plt.yticks(size=10)
plt.xlim(0, 150)
plt.ylim(0, 0.1)
plt.ylabel('Density', size=15, fontweight='bold')
plt.title("PM2.5 - House 2", size=18)
plt.legend(loc=1, prop={'size': 12})
plt.show()

In [ ]:
day12 = round(H1['PM2.5'].loc['2019-02-12 00:00:00': '2019-02-12 23:00:00'].mean(),2)
day13 = round(H1['PM2.5'].loc['2019-02-13 00:00:00': '2019-02-13 23:00:00'].mean(),2)
day14 = round(H1['PM2.5'].loc['2019-02-14 00:00:00': '2019-02-14 23:00:00'].mean(),2)
day15 = round(H1['PM2.5'].loc['2019-02-15 00:00:00': '2019-02-15 23:00:00'].mean(),2)
day16 = round(H1['PM2.5'].loc['2019-02-16 00:00:00': '2019-02-16 23:00:00'].mean(),2)
day17 = round(H1['PM2.5'].loc['2019-02-17 00:00:00': '2019-02-17 23:00:00'].mean(),2)
day19 = round(H2['PM2.5'].loc['2019-02-19 00:00:00': '2019-02-19 23:00:00'].mean(),2)
day20 = round(H2['PM2.5'].loc['2019-02-20 00:00:00': '2019-02-20 23:00:00'].mean(),2)
day21 = round(H2['PM2.5'].loc['2019-02-21 00:00:00': '2019-02-21 23:00:00'].mean(),2)
day22 = round(H2['PM2.5'].loc['2019-02-22 00:00:00': '2019-02-22 23:00:00'].mean(),2)
day23 = round(H2['PM2.5'].loc['2019-02-23 00:00:00': '2019-02-23 23:00:00'].mean(),2)
day24 = round(H2['PM2.5'].loc['2019-02-24 00:00:00': '2019-02-24 23:00:00'].mean(),2)
day25 = round(H2['PM2.5'].loc['2019-02-25 00:00:00': '2019-02-25 23:00:00'].mean(),2)


dailypm25 = [day12, day13, day14, day15, day16, day17, day19, day20, day21, day22, day23, day24, day25]

dailypm25_df = pd.Series.from_array(dailypm25)

x_labels = ['12th', '13th', '14th', '15th', '16th', '17th', '19th', '20th', '21st', '22nd', '23rd', '24th', '25th']

# Plot the figure.
plt.figure(figsize=(15,5))
ax = dailypm25_df.plot(kind='bar', color=['mediumseagreen','mediumseagreen','mediumseagreen','mediumseagreen','mediumseagreen',
                                          'mediumseagreen','orange','orange','orange','orange','orange',
                                          'orange', 'orange'], edgecolor='black')
ax.set_title('PM2.5 Daily Mean', size=18)
ax.set_ylim(0, 40)
lgd_H1 = mpatches.Patch(color='mediumseagreen', label='House 1')
lgd_H2 = mpatches.Patch(color='orange', label='House 2')
ax.legend(handles=[lgd_H1, lgd_H2], loc=1, prop={'size': 12})
ax.set_xticklabels(x_labels, size=12, rotation=0)
ax.set_yticklabels('')
ax.set_xlabel('Days', size=15, fontweight='bold')
ax.set_ylabel('PM2.5', size=15, fontweight='bold')


def add_value_labels(ax, spacing=5):
    """Add labels to the end of each bar in a bar chart.

    Arguments:
        ax (matplotlib.axes.Axes): The matplotlib object containing the axes
            of the plot to annotate.
        spacing (int): The distance between the labels and the bars.
    """

    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.1f}".format(y_value)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(0, space),          # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va,                      # Vertically align label differently for
            size=15)                    # Change text size                    



# Call the function above. All the magic happens there.
add_value_labels(ax)

In [ ]:
H1['PM2.5'].loc['2019-02-15 00:00:00': '2019-02-15 23:00:00']

In [ ]:
H1['PM2.5'].sort_values(ascending=False).head(15)

In [ ]:
#Consulting the activity record, the occupant mentioned Frying occurred at 19:15. As such, we will interpret the value from 
#18:00 as faulty and remove it.

display(H1[['PM2.5', 'Frying', 'Boiling', 'Vacuuming_KR', 'Vacuuming_LR', 'Vacuuming_BR', 'Mopping_KR', 'Mopping_LR', 'Mopping_BR',]][H1['LivingRoom'] == 1].loc['2019-02-15 11:00:00': '2019-02-15 22:00:00'])
print('LIVING ROOM')

display(H1[['PM2.5', 'Frying', 'Boiling', 'Vacuuming_KR', 'Vacuuming_LR', 'Vacuuming_BR', 'Mopping_KR', 'Mopping_LR', 'Mopping_BR',]][H1['Kitchen'] == 1].loc['2019-02-15 11:00:00': '2019-02-15 22:00:00'])
print('KITCHEN')

display(H1[['PM2.5', 'Frying', 'Boiling', 'Vacuuming_KR', 'Vacuuming_LR', 'Vacuuming_BR', 'Mopping_KR', 'Mopping_LR', 'Mopping_BR',]][H1['Bedroom'] == 1].loc['2019-02-15 11:00:00': '2019-02-15 22:00:00'])
print('BEDROOM')



In [ ]:
H2['PM2.5'].sort_values(ascending=False).head(40)

In [ ]:
display(H2[['PM2.5', 'Frying', 'Boiling', 'Vacuuming_KR', 'Vacuuming_LR', 'Vacuuming_BR', 'Mopping_KR', 'Mopping_LR', 'Mopping_BR',]][H2['LivingRoom'] == 1].loc['2019-02-23 13:00:00': '2019-02-24 00:00:00'])
print('LIVING ROOM')

display(H2[['PM2.5', 'Frying', 'Boiling', 'Vacuuming_KR', 'Vacuuming_LR', 'Vacuuming_BR', 'Mopping_KR', 'Mopping_LR', 'Mopping_BR',]][H2['Kitchen'] == 1].loc['2019-02-23 13:00:00': '2019-02-24 00:00:00'])
print('KITCHEN')

display(H2[['PM2.5', 'Frying', 'Boiling', 'Vacuuming_KR', 'Vacuuming_LR', 'Vacuuming_BR', 'Mopping_KR', 'Mopping_LR', 'Mopping_BR',]][H2['Bedroom'] == 1].loc['2019-02-23 13:00:00': '2019-02-24 00:00:00'])
print('BEDROOM')

In [ ]:
def corr_matrix(df):

    df_corr_matrix=df.corr()["PM2.5"].drop(df.corr().index[1])
    best_features_list = df_corr_matrix[abs(df_corr_matrix) >= 0.3].sort_values(ascending=False)
    weak_features_list = df_corr_matrix[(abs(df_corr_matrix) >= 0.2) & (abs(df_corr_matrix) < 0.3)].sort_values(ascending=False)

    print("There are {} moderately (> 0.3) correlated values with PM2.5:\n{}".format(len(best_features_list), best_features_list))
    print('\n')
    print("There are {} weakly (0.2 - 0.3) correlated values with PM2.5:\n{}".format(len(weak_features_list), weak_features_list))

In [ ]:
corr_matrix(LR1)

In [ ]:
corr_matrix(KR1)

In [ ]:
corr_matrix(BR1)

In [ ]:
corr_matrix(H1)

In [ ]:
corr_matrix(LR2)

In [ ]:
corr_matrix(KR2)

In [ ]:
corr_matrix(BR2)

In [ ]:
corr_matrix(H2)

In [ ]:
corr_matrix(H)

# Boxplots

Here we first compare the mean PM.25 of all 3 rooms in both houses and then we compare most predictors against PM2.5 for both houses.

In [ ]:
EDA = round(H,2).copy()
EDA1 = round(H1,2).copy()
EDA2 = round(H2,2).copy()

EDA.name = 'EDA'
EDA1.name = 'EDA1'
EDA2.name = 'EDA2'

In [ ]:
EDA['TVOC_cut'] = pd.cut(EDA['TVOC'], 4)
EDA['TVOC_Qcut'] = pd.qcut(EDA['TVOC'], 4)

EDA['CO2_cut'] = pd.cut(EDA['CO2'], 4)
EDA['CO2_Qcut'] = pd.qcut(EDA['CO2'], 4)

EDA['Temp_cut'] = pd.cut(EDA['Temp'], 4)
EDA['Temp_Qcut'] = pd.qcut(EDA['Temp'], 4)

EDA['RH_cut'] = pd.cut(EDA['RH'], 4)
EDA['RH_Qcut'] = pd.qcut(EDA['RH'], 4)

EDA['AirPres_cut'] = pd.cut(EDA['AirPres'], 4)
EDA['AirPres_Qcut'] = pd.qcut(EDA['AirPres'], 4)

EDA['Room'] = 'Kitchen'
EDA['Room'][EDA['LivingRoom'] == 1] = 'Living Room'
EDA['Room'][EDA['Bedroom'] == 1] = 'Bedroom'

EDA['timeOfDay'] = EDA['timeOfDay'].map({0:'00:00 - 06:00', 1:'06:00 - 12:00', 2:'12:00 - 18:00', 3:'18:00 - 00:00'})


EDA1['TVOC_cut'] = pd.cut(EDA1['TVOC'], 4)
EDA1['TVOC_Qcut'] = pd.qcut(EDA1['TVOC'], 4)

EDA1['CO2_cut'] = pd.cut(EDA1['CO2'], 4)
EDA1['CO2_Qcut'] = pd.qcut(EDA1['CO2'], 4)

EDA1['Temp_cut'] = pd.cut(EDA1['Temp'], 4)
EDA1['Temp_Qcut'] = pd.qcut(EDA1['Temp'], 4)

EDA1['RH_cut'] = pd.cut(EDA1['RH'], 4)
EDA1['RH_Qcut'] = pd.qcut(EDA1['RH'], 4)

EDA1['AirPres_cut'] = pd.cut(EDA1['AirPres'], 4)
EDA1['AirPres_Qcut'] = pd.qcut(EDA1['AirPres'], 4)

EDA1['Room'] = 'Kitchen'
EDA1['Room'][EDA1['LivingRoom'] == 1] = 'Living Room'
EDA1['Room'][EDA1['Bedroom'] == 1] = 'Bedroom'

EDA1['timeOfDay'] = EDA1['timeOfDay'].map({0:'00:00 - 06:00', 1:'06:00 - 12:00', 2:'12:00 - 18:00', 3:'18:00 - 00:00'})


EDA2['TVOC_cut'] = pd.cut(EDA2['TVOC'], 4)
EDA2['TVOC_Qcut'] = pd.qcut(EDA2['TVOC'], 4)

EDA2['CO2_cut'] = pd.cut(EDA2['CO2'], 4)
EDA2['CO2_Qcut'] = pd.qcut(EDA2['CO2'], 4)

EDA2['Temp_cut'] = pd.cut(EDA2['Temp'], 4)
EDA2['Temp_Qcut'] = pd.qcut(EDA2['Temp'], 4)

EDA2['RH_cut'] = pd.cut(EDA2['RH'], 4)
EDA2['RH_Qcut'] = pd.qcut(EDA2['RH'], 4)

EDA2['AirPres_cut'] = pd.cut(EDA2['AirPres'], 4)
EDA2['AirPres_Qcut'] = pd.qcut(EDA2['AirPres'], 4)

EDA2['Room'] = 'Kitchen'
EDA2['Room'][EDA2['LivingRoom'] == 1] = 'Living Room'
EDA2['Room'][EDA2['Bedroom'] == 1] = 'Bedroom'

EDA2['timeOfDay'] = EDA2['timeOfDay'].map({0:'00:00 - 06:00', 1:'06:00 - 12:00', 2:'12:00 - 18:00', 3:'18:00 - 00:00'})



xEDA1 = EDA1.copy()

xEDA1[['Temp', 'RH', 'PM2.5', 'TVOC', 'CO2', 'AirPres','MaxOccupancy_BR', 
     'MaxOccupancy_LR','MaxOccupancy_KR']] = MinMaxScaler().fit_transform(
     xEDA1[['Temp', 'RH', 'PM2.5', 'TVOC', 'CO2', 'AirPres', 'MaxOccupancy_BR', 
             'MaxOccupancy_LR','MaxOccupancy_KR']])

xEDA1 = xEDA1.sort_index()
xEDA1.name = 'xEDA1'



xEDA2 = EDA2.copy()

xEDA2[['Temp', 'RH', 'PM2.5', 'TVOC', 'CO2', 'AirPres','MaxOccupancy_BR', 
     'MaxOccupancy_LR','MaxOccupancy_KR']] = MinMaxScaler().fit_transform(
     xEDA2[['Temp', 'RH', 'PM2.5', 'TVOC', 'CO2', 'AirPres', 'MaxOccupancy_BR', 
             'MaxOccupancy_LR','MaxOccupancy_KR']])

xEDA2 = xEDA2.sort_index()
xEDA2.name = 'xEDA2'

In [ ]:
def my_boxplot(df, feature):

    plt.figure(figsize=(15,5))
    sns.boxplot(x='Room', y='PM2.5', hue= feature, data=df)
    plt.xticks(np.arange(3), size=15, fontweight='bold')
    plt.xlabel('')
    plt.ylabel('PM2.5', size=15, fontweight='bold')
    plt.yticks(size=12)

    if df.name == 'EDA1':
        plt.title("PM2.5 - "+feature+' - House 1', size=18)
    else:
        plt.title("PM2.5 - "+feature+' - House 2', size=18)  
    plt.legend(loc=1, prop={'size': 15})
    plt.show();

    display(df.groupby(['Room',feature],sort=True)['PM2.5'].agg(['count', 'mean', 'median']).round(2))


def my_scatterplot(house, feature, xlim_min, xlim_max):

    plt.figure(figsize=(15,5))
    sns.scatterplot(x=feature, y='PM2.5', hue='Room', hue_order = ['Kitchen', 'Living Room', 'Bedroom'], data=EDA[EDA['House'] == house])
    plt.xlabel(feature, size=15, fontweight='bold')
    plt.ylabel('PM2.5', size=15, fontweight='bold')
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.xlim(xlim_min, xlim_max)
    plt.ylim(0, 139)
    plt.title("PM2.5 ~ "+feature+' - House '+str(house), size=18)
    plt.legend(loc=1, prop={'size': 15})
    plt.show();


def my_stepplot(df, room, feature):
    
    plt.figure(figsize=(15, 5))
    plt.plot(df["PM2.5"][df['Room'] == room])
    plt.ylim(0,1.13)
    plt.ylabel('Normalized PM2.5', size=15, fontweight='bold')
    plt.xlabel('Days', size=15, fontweight='bold')
    
    if df.name == 'xEDA1':
        plt.title('House 1 - ' + room + ' - PM2.5 ~ ' + feature, size=18)
    else:
        plt.title('House 2 - ' + room + ' - PM2.5 ~ ' + feature, size=18)        

    plt.fill_between(df.index,df[feature], step="pre", alpha=0.3, color='darkorange')
    event = mpatches.Patch(color='darkorange', label=feature+' active (On/Yes/Open)', alpha=0.3)
    plt.legend(handles=[event], loc=1, prop={'size': 15})
    plt.show();

In [ ]:
my_stepplot(xEDA1, 'Living Room', 'Frying')

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x='House', y='PM2.5', hue='Room', hue_order = ['Kitchen', 'Living Room', 'Bedroom'], data=EDA)
plt.xticks(np.arange(2),('House 1', 'House 2'), size=15, fontweight='bold')
plt.xlabel('')
plt.ylabel('PM2.5', size=15, fontweight='bold')
plt.yticks(size=12)
plt.ylim(0,159)
plt.title("PM2.5 - Overview", size=18)
plt.legend(prop={'size': 15})
plt.show()

House 2 has considerably more outliers than 1. Given their rather different room dimensions, it could explain why most of the Kitchen and Living Room outliers didn't seep into the Bedroom.

House 2 has 4 features that House 1 doesn't have; shower, LR_OpenDoor, KR_OpenDoor and BR_OpenDoor. We'll look at these first.

We weren't able to see any noticeable differences in the level of PM with the activity of Shower, as this mostly would affect a room we are not tracking and we only have 9 observations out of 139 where Shower is active.

The only room with a considerable count of both "1"s and "0"s is the Bedroom and there doesn't seem to be any difference in PM2.5 between them. 

We'll now look into some of the continuous variables from our sensors.

In [ ]:
my_scatterplot(1, 'TVOC', -20, 1125)
my_boxplot(EDA1, 'TVOC_cut')
my_boxplot(EDA1, 'TVOC_Qcut')

my_scatterplot(2, 'TVOC', 95, 1000)
my_boxplot(EDA2, 'TVOC_cut')
my_boxplot(EDA2, 'TVOC_Qcut')

In [ ]:
my_scatterplot(1, 'CO2', 350, 1475)
my_boxplot(EDA1, 'CO2_cut')
my_boxplot(EDA1, 'CO2_Qcut')

my_scatterplot(2, 'CO2', 350, 3600)
my_boxplot(EDA2, 'CO2_cut')
my_boxplot(EDA2, 'CO2_Qcut')

In [ ]:
my_scatterplot(1, 'Temp', 14.8, 23)
my_boxplot(EDA1, 'Temp_cut')
my_boxplot(EDA1, 'Temp_Qcut')

my_scatterplot(2, 'Temp', 19, 25)
my_boxplot(EDA2, 'Temp_cut')
my_boxplot(EDA2, 'Temp_Qcut')

In [ ]:
my_scatterplot(1, 'RH', 50, 77)
my_boxplot(EDA1, 'RH_cut')
my_boxplot(EDA1, 'RH_Qcut')

my_scatterplot(2, 'RH', 50, 77)
my_boxplot(EDA2, 'RH_cut')
my_boxplot(EDA2, 'RH_Qcut')

In [ ]:
my_scatterplot(1, 'AirPres', 997, 1030)
my_boxplot(EDA1, 'AirPres_cut')
my_boxplot(EDA1, 'AirPres_Qcut')

my_scatterplot(2, 'AirPres', 997, 1030)
my_boxplot(EDA2, 'AirPres_cut')
my_boxplot(EDA2, 'AirPres_Qcut')

### We now plot all of our categorical variables against our target variable for both houses.

In [ ]:
#Reference for variables
display(EDA1.columns)
display(EDA2.columns)

In [ ]:
my_boxplot(EDA2, 'Shower')
my_stepplot(xEDA2,'Living Room', 'Shower')
my_stepplot(xEDA2,'Kitchen', 'Shower')
my_stepplot(xEDA2,'Bedroom', 'Shower')

my_boxplot(EDA2, 'Door_KR')
my_stepplot(xEDA2,'Living Room', 'Door_KR')
my_stepplot(xEDA2,'Kitchen', 'Door_KR')
my_stepplot(xEDA2,'Bedroom', 'Door_KR')

my_boxplot(EDA2, 'Door_LR')
my_stepplot(xEDA2,'Living Room', 'Door_LR')
my_stepplot(xEDA2,'Kitchen', 'Door_LR')
my_stepplot(xEDA2,'Bedroom', 'Door_LR')

my_boxplot(EDA2, 'Door_BR')
my_stepplot(xEDA2,'Living Room', 'Door_BR')
my_stepplot(xEDA2,'Kitchen', 'Door_BR')
my_stepplot(xEDA2,'Bedroom', 'Door_BR')

In [ ]:
my_boxplot(EDA1, 'MaxOccupancy_LR')
my_stepplot(xEDA1,'Living Room', 'MaxOccupancy_LR')
my_stepplot(xEDA1,'Kitchen', 'MaxOccupancy_LR')
my_stepplot(xEDA1,'Bedroom', 'MaxOccupancy_LR')

my_boxplot(EDA1, 'MaxOccupancy_KR')
my_stepplot(xEDA1,'Living Room', 'MaxOccupancy_KR')
my_stepplot(xEDA1,'Kitchen', 'MaxOccupancy_KR')
my_stepplot(xEDA1,'Bedroom', 'MaxOccupancy_KR')

my_boxplot(EDA1, 'MaxOccupancy_BR')
my_stepplot(xEDA1,'Living Room', 'MaxOccupancy_BR')
my_stepplot(xEDA1,'Kitchen', 'MaxOccupancy_BR')
my_stepplot(xEDA1,'Bedroom', 'MaxOccupancy_BR')

In [ ]:
my_boxplot(EDA2, 'MaxOccupancy_LR')
my_stepplot(xEDA2,'Living Room', 'MaxOccupancy_LR')
my_stepplot(xEDA2,'Kitchen', 'MaxOccupancy_LR')
my_stepplot(xEDA2,'Bedroom', 'MaxOccupancy_LR')

my_boxplot(EDA2, 'MaxOccupancy_KR')
my_stepplot(xEDA2,'Living Room', 'MaxOccupancy_KR')
my_stepplot(xEDA2,'Kitchen', 'MaxOccupancy_KR')
my_stepplot(xEDA2,'Bedroom', 'MaxOccupancy_KR')

my_boxplot(EDA2, 'MaxOccupancy_BR')
my_stepplot(xEDA2,'Living Room', 'MaxOccupancy_BR')
my_stepplot(xEDA2,'Kitchen', 'MaxOccupancy_BR')
my_stepplot(xEDA2,'Bedroom', 'MaxOccupancy_BR')

In [ ]:
my_boxplot(EDA1, 'Window_LR')
my_stepplot(xEDA1,'Living Room', 'Window_LR')
my_stepplot(xEDA1,'Kitchen', 'Window_LR')
my_stepplot(xEDA1,'Bedroom', 'Window_LR')

my_boxplot(EDA1, 'Window_BR')
my_stepplot(xEDA1,'Living Room', 'Window_BR')
my_stepplot(xEDA1,'Kitchen', 'Window_BR')
my_stepplot(xEDA1,'Bedroom', 'Window_BR')

In [ ]:
my_boxplot(EDA2, 'Window_LR')
my_stepplot(xEDA2,'Living Room', 'Window_LR')
my_stepplot(xEDA2,'Kitchen', 'Window_LR')
my_stepplot(xEDA2,'Bedroom', 'Window_LR')

my_boxplot(EDA2, 'Window_BR')
my_stepplot(xEDA2,'Living Room', 'Window_BR')
my_stepplot(xEDA2,'Kitchen', 'Window_BR')
my_stepplot(xEDA2,'Bedroom', 'Window_BR')

In [ ]:
my_boxplot(EDA1, 'Curtains_LR')
my_stepplot(xEDA1,'Living Room', 'Curtains_LR')
my_stepplot(xEDA1,'Kitchen', 'Curtains_LR')
my_stepplot(xEDA1,'Bedroom', 'Curtains_LR')

my_boxplot(EDA1, 'Curtains_KR')
my_stepplot(xEDA1,'Living Room', 'Curtains_KR')
my_stepplot(xEDA1,'Kitchen', 'Curtains_KR')
my_stepplot(xEDA1,'Bedroom', 'Curtains_KR')

my_boxplot(EDA1, 'Curtains_BR')
my_stepplot(xEDA1,'Living Room', 'Curtains_BR')
my_stepplot(xEDA1,'Kitchen', 'Curtains_BR')
my_stepplot(xEDA1,'Bedroom', 'Curtains_BR')

In [ ]:
my_boxplot(EDA2, 'Curtains_LR')
my_stepplot(xEDA2,'Living Room', 'Curtains_LR')
my_stepplot(xEDA2,'Kitchen', 'Curtains_LR')
my_stepplot(xEDA2,'Bedroom', 'Curtains_LR')

my_boxplot(EDA2, 'Curtains_BR')
my_stepplot(xEDA2,'Living Room', 'Curtains_BR')
my_stepplot(xEDA2,'Kitchen', 'Curtains_BR')
my_stepplot(xEDA2,'Bedroom', 'Curtains_BR')

In [ ]:
my_boxplot(EDA2, 'Vacuuming_LR')
my_stepplot(xEDA2,'Living Room', 'Vacuuming_LR')
my_stepplot(xEDA2,'Kitchen', 'Vacuuming_LR')
my_stepplot(xEDA2,'Bedroom', 'Vacuuming_LR')

my_boxplot(EDA2, 'Vacuuming_KR')
my_stepplot(xEDA2,'Living Room', 'Vacuuming_KR')
my_stepplot(xEDA2,'Kitchen', 'Vacuuming_KR')
my_stepplot(xEDA2,'Bedroom', 'Vacuuming_KR')

my_boxplot(EDA2, 'Vacuuming_BR')
my_stepplot(xEDA2,'Living Room', 'Vacuuming_BR')
my_stepplot(xEDA2,'Kitchen', 'Vacuuming_BR')
my_stepplot(xEDA2,'Bedroom', 'Vacuuming_BR')

In [ ]:
my_boxplot(EDA1, 'Mopping_KR')
my_stepplot(xEDA1,'Living Room', 'Mopping_KR')
my_stepplot(xEDA1,'Kitchen', 'Mopping_KR')
my_stepplot(xEDA1,'Bedroom', 'Mopping_KR')

In [ ]:
my_boxplot(EDA2, 'Mopping_LR')
my_stepplot(xEDA2,'Living Room', 'Mopping_LR')
my_stepplot(xEDA2,'Kitchen', 'Mopping_LR')
my_stepplot(xEDA2,'Bedroom', 'Mopping_LR')

my_boxplot(EDA2, 'Mopping_KR')
my_stepplot(xEDA2,'Living Room', 'Mopping_KR')
my_stepplot(xEDA2,'Kitchen', 'Mopping_KR')
my_stepplot(xEDA2,'Bedroom', 'Mopping_KR')

my_boxplot(EDA2, 'Mopping_BR')
my_stepplot(xEDA2,'Living Room', 'Mopping_BR')
my_stepplot(xEDA2,'Kitchen', 'Mopping_BR')
my_stepplot(xEDA2,'Bedroom', 'Mopping_BR')

In [ ]:
my_boxplot(EDA1, 'Heating_LR')
my_stepplot(xEDA1,'Living Room', 'Heating_LR')
my_stepplot(xEDA1,'Kitchen', 'Heating_LR')
my_stepplot(xEDA1,'Bedroom', 'Heating_LR')

my_boxplot(EDA1, 'Heating_KR')
my_stepplot(xEDA1,'Living Room', 'Heating_KR')
my_stepplot(xEDA1,'Kitchen', 'Heating_KR')
my_stepplot(xEDA1,'Bedroom', 'Heating_KR')

my_boxplot(EDA1, 'Heating_BR')
my_stepplot(xEDA1,'Living Room', 'Heating_BR')
my_stepplot(xEDA1,'Kitchen', 'Heating_BR')
my_stepplot(xEDA1,'Bedroom', 'Heating_BR')

In [ ]:
my_boxplot(EDA2, 'Heating_LR')
my_stepplot(xEDA2,'Living Room', 'Heating_LR')
my_stepplot(xEDA2,'Kitchen', 'Heating_LR')
my_stepplot(xEDA2,'Bedroom', 'Heating_LR')

my_boxplot(EDA2, 'Heating_KR')
my_stepplot(xEDA2,'Living Room', 'Heating_LR')
my_stepplot(xEDA2,'Kitchen', 'Heating_LR')
my_stepplot(xEDA2,'Bedroom', 'Heating_LR')

my_boxplot(EDA2, 'Heating_BR')
my_stepplot(xEDA2,'Living Room', 'Heating_BR')
my_stepplot(xEDA2,'Kitchen', 'Heating_BR')
my_stepplot(xEDA2,'Bedroom', 'Heating_BR')

In [ ]:
my_boxplot(EDA1, 'Frying')
my_stepplot(xEDA1,'Living Room', 'Frying')
my_stepplot(xEDA1,'Kitchen', 'Frying')
my_stepplot(xEDA1,'Bedroom', 'Frying')

my_boxplot(EDA2, 'Frying')
my_stepplot(xEDA2,'Living Room', 'Frying')
my_stepplot(xEDA2,'Kitchen', 'Frying')
my_stepplot(xEDA2,'Bedroom', 'Frying')

In [ ]:
my_boxplot(EDA1, 'Boiling')
my_stepplot(xEDA1,'Living Room', 'Boiling')
my_stepplot(xEDA1,'Kitchen', 'Boiling')
my_stepplot(xEDA1,'Bedroom', 'Boiling')

my_boxplot(EDA2, 'Boiling')
my_stepplot(xEDA2,'Living Room', 'Boiling')
my_stepplot(xEDA2,'Kitchen', 'Boiling')
my_stepplot(xEDA2,'Bedroom', 'Boiling')

In [ ]:
my_boxplot(EDA1, 'Dishwasher')
my_stepplot(xEDA1,'Living Room', 'Dishwasher')
my_stepplot(xEDA1,'Kitchen', 'Dishwasher')
my_stepplot(xEDA1,'Bedroom', 'Dishwasher')

my_boxplot(EDA2, 'Dishwasher')
my_stepplot(xEDA2,'Living Room', 'Dishwasher')
my_stepplot(xEDA2,'Kitchen', 'Dishwasher')
my_stepplot(xEDA2,'Bedroom', 'Dishwasher')

In [ ]:
my_boxplot(EDA2, 'WashingMachine')
my_stepplot(xEDA2,'Living Room', 'WashingMachine')
my_stepplot(xEDA2,'Kitchen', 'WashingMachine')
my_stepplot(xEDA2,'Bedroom', 'WashingMachine')

In [ ]:
my_boxplot(EDA2, 'Toaster')
my_stepplot(xEDA2,'Living Room', 'Toaster')
my_stepplot(xEDA2,'Kitchen', 'Toaster')
my_stepplot(xEDA2,'Bedroom', 'Toaster')

In [ ]:
my_boxplot(EDA1, 'FireLit')
my_stepplot(xEDA1,'Living Room', 'FireLit')
my_stepplot(xEDA1,'Kitchen', 'FireLit')
my_stepplot(xEDA1,'Bedroom', 'FireLit')

In [ ]:
my_boxplot(EDA1, 'timeOfDay')
my_boxplot(EDA2, 'timeOfDay')

In [ ]:
#Not using the function as we need to make some adjustments to the legend size

plt.figure(figsize=(15,5))
sns.boxplot(x='Room', y='PM2.5', hue= 'Hour', data=EDA1)
plt.xticks(np.arange(3), size=15, fontweight='bold')
plt.xlabel('')
plt.ylabel('PM2.5', size=15, fontweight='bold')
plt.yticks(size=12)

plt.title("PM2.5 - Hour - House 1", size=18)
plt.legend().remove()
plt.show()

display(EDA1.groupby(['Room','Hour'],sort=True)['PM2.5'].agg(['count', 'mean', 'median']).round(2))

plt.figure(figsize=(15,5))
sns.boxplot(x='Room', y='PM2.5', hue= 'Hour', data=EDA2)
plt.xticks(np.arange(3), size=15, fontweight='bold')
plt.xlabel('')
plt.ylabel('PM2.5', size=15, fontweight='bold')
plt.yticks(size=12)

plt.title("PM2.5 - Hour - House 2", size=18)
plt.legend().remove()
plt.show()

display(EDA2.groupby(['Room','Hour'],sort=True)['PM2.5'].agg(['count', 'mean', 'median']).round(2))


# Modeling

In [ ]:
H = H.drop(['Hour', 'House'], axis=1)
H1 = H1.drop(['Hour', 'House'], axis=1)
H2 = H2.drop(['Hour', 'House'], axis=1)

In [ ]:
def train_test (df):
        
    X = df.drop(["PM2.5"], axis=1)
    y = df["PM2.5"]
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)
            
    return (X_train, X_test, y_train, y_test)

In [ ]:
def rmse(y_pred, y_test):

    return np.sqrt(np.mean((y_pred - y_test)**2))

In [ ]:
def rf_baseline(df):

    X_train, X_test, y_train, y_test = train_test(df)
        
    model = RandomForestRegressor(random_state = 42)
    model.fit(X_train, y_train);
    y_pred = model.predict(X_test)

    print('Baseline RMSE:',round(rmse(y_pred, y_test),4))
    print('Baseline MAE:',round(mean_absolute_error(y_pred, y_test),4))

In [ ]:
def rf_baseline(df):
    
    X_train, X_test, y_train, y_test = train_test(df)
    

    max_features = ['auto', None]


    params = {'max_features': max_features}
    

    scorers = {'RMSE': make_scorer(rmse, greater_is_better=False),
               'MAE': make_scorer(mean_absolute_error, greater_is_better=False)}

    cv = RepeatedKFold(n_splits=10, n_repeats=10)
    random.seed(42)
    grid = GridSearchCV(estimator=RandomForestRegressor(random_state = 42),
                        param_grid=params, 
                        verbose=1, 
                        cv=cv, 
                        n_jobs =-1, 
                        scoring=scorers, 
                        refit = 'RMSE')
    
    grid = grid.fit(X_train, y_train)
    
    y_pred = grid.predict(X_test)
    
    print('Parameters used:', grid.best_params_,'\n')
    print('Cross Validation:')
    print('RMSE Score:', round(-1*grid.best_score_,4))
    print('MAE Score:',round(grid.cv_results_['mean_test_MAE'][grid.cv_results_['rank_test_MAE'] == 1].item(0)*-1,4))
    print ('\nHoldout set:')
    print('RMSE Score:',round(-1*grid.score(X_test, y_test),4))
    print('MAE Score:',round(mean_absolute_error(y_pred, y_test),4)) 

In [ ]:
rf_baseline(H)

In [ ]:
print('Dataset: H1')
rf_baseline(H1)
print('\n')
print('Dataset: H2')
rf_baseline(H2)
print('\n')
print('Dataset: H')
rf_baseline(H)
print('\n')
print('Dataset: LR1')
rf_baseline(LR1)
print('\n')
print('Dataset: small_LR1')
rf_baseline(small_LR1)
print('\n')
print('Dataset: KR1')
rf_baseline(KR1)
print('\n')
print('Dataset: small_KR1')
rf_baseline(small_KR1)
print('\n')
print('Dataset: BR1')
rf_baseline(BR1)
print('\n')
print('Dataset: small_BR1')
rf_baseline(small_BR1)
print('\n')
print('Dataset: LR2')
rf_baseline(LR2)
print('\n')
print('Dataset: small_LR2')
rf_baseline(small_LR2)
print('\n')
print('Dataset: KR2')
rf_baseline(KR2)
print('\n')
print('Dataset: small_KR2')
rf_baseline(small_KR2)
print('\n')
print('Dataset: BR2')
rf_baseline(BR2)
print('\n')
print('Dataset: small_BR2')
rf_baseline(small_BR2)

In [ ]:
[int(x) for x in np.logspace(start = np.log10(10), stop = np.log10(300), num = 20)]

In [ ]:
def find_rf_params(df):

    X_train, X_test, y_train, y_test = train_test(df)

    n_estimators = [int(x) for x in np.logspace(start = np.log10(100), stop = np.log10(1000), num = 50)]
    max_features = ['auto', 'sqrt', 'log2', None]   
    max_depth = [int(x) for x in np.logspace(start = np.log10(10), stop = np.log10(300), num = 50)]
    max_depth.append(None)
    min_samples_split = [2, 3]
    min_samples_leaf = [1, 2, 3]
    bootstrap = [True, False]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

    randomgrid = RandomizedSearchCV(estimator = RandomForestRegressor(random_state=random.seed(42)), 
                                   param_distributions = random_grid, 
                                   n_iter = 1000, 
                                   cv = 10, 
                                   verbose=1, 
                                   random_state=random.seed(42), 
                                   n_jobs = -1,
                                   scoring=make_scorer(rmse, greater_is_better=False))  
    
    randomgrid.fit(X_train, y_train)
    
    return randomgrid

In [ ]:
grid1 = find_rf_params(H1)
#grid2 = find_rf_params(H2)
#grid3 = find_rf_params(H)

#grid4 = find_rf_params(LR1)
#grid5 = find_rf_params(KR1)
#grid6 = find_rf_params(BR1)
#grid7 = find_rf_params(LR2)
#grid8 = find_rf_params(KR2)
#grid9 = find_rf_params(BR2)

#grid10 = find_rf_params(small_LR1)
#grid11 = find_rf_params(small_KR1)
#grid12 = find_rf_params(small_BR1)
#grid13 = find_rf_params(small_LR2)
#grid14 = find_rf_params(small_KR2)
#grid15 = find_rf_params(small_BR2)

In [ ]:
[int(x) for x in np.logspace(start = np.log10(10), stop = np.log10(60), num = 10)]

In [ ]:
print('----- H1 -----')
print(grid1.best_params_)
print('----- H2 -----')
print(grid2.best_params_)
print('----- H -----')
print(grid3.best_params_)

In [ ]:
print('----- H1 -----')
print(grid1.best_params_)
print('----- H2 -----')
print(grid2.best_params_)
print('----- H -----')
print(grid3.best_params_)
print('----- LR1 -----')
print(grid4.best_params_)
print('----- KR1 -----')
print(grid5.best_params_)
print('----- BR1 -----')
print(grid6.best_params_)
print('----- LR2 -----')
print(grid7.best_params_)
print('----- KR2 -----')
print(grid8.best_params_)
print('----- BR2 -----')
print(grid9.best_params_)
print('----- small_LR1 -----')
print(grid10.best_params_)
print('----- small_KR1 -----')
print(grid11.best_params_)
print('----- small_BR1 -----')
print(grid12.best_params_)
print('----- small_LR2 -----')
print(grid13.best_params_)
print('----- small_KR2 -----')
print(grid14.best_params_)
print('----- small_BR2 -----')
print(grid15.best_params_)

In [ ]:
def rf(df):
    
    X_train, X_test, y_train, y_test = train_test(df)
    

    max_features = ['sqrt', 'auto']
    max_depth = [int(x) for x in np.logspace(start = np.log10(10), stop = np.log10(60), num = 10)]


    params = {'max_features': max_features,
              'max_depth': max_depth}
    

    scorers = {'RMSE': make_scorer(rmse, greater_is_better=False),
               'MAE': make_scorer(mean_absolute_error, greater_is_better=False)}

    cv = RepeatedKFold(n_splits=10, n_repeats=10)
        
    grid = GridSearchCV(estimator=RandomForestRegressor(random_state = random.seed(42),
                                                        n_estimators = 1000, 
                                                        min_samples_split = 2,
                                                        min_samples_leaf = 1,
                                                        bootstrap = False),
                        param_grid=params, 
                        verbose=1, 
                        cv=cv, 
                        n_jobs =-1, 
                        scoring=scorers, 
                        refit = 'RMSE')
    
    grid = grid.fit(X_train, y_train)
    
    y_pred = grid.predict(X_test)
    
    print('Parameters used:', grid.best_params_,'\n')
    print('Cross Validation:')
    print('RMSE Score:', round(-1*grid.best_score_,4))
    print('MAE Score:',round(grid.cv_results_['mean_test_MAE'][grid.cv_results_['rank_test_MAE'] == 1].item(0)*-1,4))
    print ('\nHoldout set:')
    print('RMSE Score:',round(-1*grid.score(X_test, y_test),4))
    print('MAE Score:',round(mean_absolute_error(y_pred, y_test),4))  
    
    feature_list = list(X_train.columns)
    importances = list(grid.best_estimator_.feature_importances_)
    feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(feature_list, importances)]
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    [print('{:18}{}'.format(*pair)) for pair in feature_importances[0:10]];
    ranked_features, values = zip(*feature_importances)

    plt.figure(figsize=(15,5))
    x_values = list(range(len((importances))))
    plt.bar(x_values, values, orientation = 'vertical')
    plt.xticks(x_values, ranked_features, rotation=90, size=12)
    plt.ylabel('Feature Importance (proportion)', size=15)
    plt.xlabel('')
    plt.title('Feature Importance')
    plt.show();

    # Extract a tree
    tree = grid.best_estimator_.estimators_[4]
    # Save the tree as a png image
    export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, class_names = y_test.name, rounded = True, proportion = False, precision = 1, filled = True)
    (graph, ) = pydot.graph_from_dot_file('C:/Users/Tom/Documents/Course/Semester 3/Project/Data Sets/tree.dot')
    graph.write_jpeg('tree.jpeg');
    display(Image(filename = 'tree.jpeg', width=600, unconfined=True))

In [ ]:
grid.best_params_

In [ ]:
rf(H1)

In [ ]:
rf(H2)

In [ ]:
rf(H)

In [ ]:
rf(LR1)

In [ ]:
rf(BR1)

In [ ]:
rf(LR2)

In [ ]:
rf(KR2)

In [ ]:
rf(BR2)

In [ ]:
rf(small_LR1)

In [ ]:
rf(small_KR1)

In [ ]:
rf(small_BR1)

In [ ]:
rf(small_LR2)

In [ ]:
rf(small_KR2)

In [ ]:
rf(small_BR2)

# Support Vector Machines - Regression

In [ ]:
def find_svr_params(df):

    X_train, X_test, y_train, y_test = train_test(df)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform (X_test) 

    C = np.logspace(np.log10(0.001), np.log10(10000), num=50)
    epsilon = np.logspace(np.log10(0.0001), np.log10(100), num=50)


    random_grid = {'C': C,
                   'epsilon': epsilon,}
    
    cv = RepeatedKFold(n_splits=10, n_repeats=10)    
    
    rf_random = RandomizedSearchCV(estimator = SVR(kernel='rbf', gamma='scale'), 
                                   param_distributions = random_grid, 
                                   n_iter = 1000, 
                                   cv = cv, 
                                   verbose=1, 
                                   random_state=random.seed(42), 
                                   n_jobs = -1,
                                   scoring=make_scorer(rmse, greater_is_better=False))    
    
    rf_random.fit(X_train, y_train)
    
    return rf_random.best_params_

In [ ]:
find_svr_params(H1)

In [ ]:
svrgrid1 = find_svr_params(H1)
svrgrid2 = find_svr_params(H2)
svrgrid3 = find_svr_params(H)

svrgrid4 = find_svr_params(LR1)
svrgrid5 = find_svr_params(KR1)
svrgrid6 = find_svr_params(BR1)
svrgrid7 = find_svr_params(LR2)
svrgrid8 = find_svr_params(KR2)
svrgrid9 = find_svr_params(BR2)

svrgrid10 = find_svr_params(small_LR1)
svrgrid11 = find_svr_params(small_KR1)
grid12 = find_svr_params(small_BR1)
svrgrid13 = find_svr_params(small_LR2)
svrgrid14 = find_svr_params(small_KR2)
svrgrid15 = find_svr_params(small_BR2)

In [ ]:
print('----- H1 -----')
print(SVRgrid1)

print('----- H2 -----')
print(SVRgrid2)

print('----- H -----')
print(SVRgrid3)

print('----- LR1 -----')
print(SVRgrid4)

print('----- KR1 -----')
print(SVRgrid5)

print('----- BR1 -----')
print(SVRgrid6)

print('----- LR2 -----')
print(SVRgrid7)

print('----- KR2 -----')
print(SVRgrid8)

print('----- BR2 -----')
print(SVRgrid9)

print('----- small_LR1 -----')
print(SVRgrid10)

print('----- small_KR1 -----')
print(SVRgrid11)

print('----- small_BR1 -----')
print(SVRgrid12)

print('----- small_LR2 -----')
print(SVRgrid13)

print('----- small_KR2 -----')
print(SVRgrid14)

print('----- small_BR2 -----')
print(SVRgrid15)


In [ ]:
def svr(df):

    X_train, X_test, y_train, y_test = train_test(df)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform (X_test) 
    
    C = np.logspace(np.log10(0.001), np.log10(10000), num=20)
    epsilon = np.logspace(np.log10(0.0001), np.log10(100), num=20)


    params = {#'C': C,
                   'epsilon': epsilon,}    
    

    scorers = {'RMSE': make_scorer(rmse, greater_is_better=False),
               'MAE': make_scorer(mean_absolute_error, greater_is_better=False)}
    
    cv = RepeatedKFold(n_splits=10, n_repeats=10)
    random.seed(42)
    grid = GridSearchCV(estimator=SVR(kernel='rbf',  gamma='scale', C=784.7599703514607),
                        param_grid=params, 
                        verbose=1, 
                        cv=cv, 
                        n_jobs =-1,
                        scoring=scorers,
                        refit='RMSE')
    
    grid = grid.fit(X_train, y_train)
    
    y_pred = grid.predict(X_test)
    
    print('Parameters used:', grid.best_params_,'\n')
    print('Cross Validation:')
    print('RMSE Score:', round(-1*grid.best_score_,4))
    print('MAE Score:',round(grid.cv_results_['mean_test_MAE'][grid.cv_results_['rank_test_MAE'] == 1].item(0)*-1,4))
    print ('\nHoldout set:')
    print('RMSE Score:',round(-1*grid.score(X_test, y_test),4))
    print('MAE Score:',round(mean_absolute_error(y_pred, y_test),4))  

In [ ]:
svr(H1)

In [ ]:
svr(H2)

In [ ]:
svr(H)

In [ ]:
svr(LR1)

In [ ]:
svr(KR1)

In [ ]:
svr(BR1)

In [ ]:
svr(LR2)

In [ ]:
svr(KR2)

In [ ]:
svr(BR2)

In [ ]:
svr(small_LR1)

In [ ]:
svr(small_KR1)

In [ ]:
svr(small_BR1)

In [ ]:
svr(small_LR2)

In [ ]:
svr(small_KR2)

In [ ]:
svr(small_BR2)

# Neural Networks

In [ ]:
def find_ann_params(df):

    X_train, X_test, y_train, y_test = train_test(df)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform (X_test) 

    #hidden_layer_sizes = 
    activation = ['identity', 'logistic', 'tanh', 'relu']
    batch_size = [int(x) for x in np.logspace(start = np.log10(10), stop = np.log10(df.shape[0]), num = 100)]
    learning_rate = ['constant','invscaling', 'adaptive']
    learning_rate_init = np.logspace(np.log10(0.0001), np.log10(0.1), num=100)
    momentum = np.logspace(np.log10(0.01), np.log10(1), num=100)
    
    
    scorers = {'RMSE': make_scorer(rmse, greater_is_better=False)}

    random_grid = {'activation': activation,
                   'batch_size' : batch_size,
                   'learning_rate' : learning_rate,
                   'learning_rate_init' : learning_rate_init,
                   'momentum' : momentum
                  }
      
    ann_random = RandomizedSearchCV(estimator = MLPRegressor(solver = 'adam'), 
                                   param_distributions = random_grid, 
                                   n_iter = 100, 
                                   cv = 10, 
                                   verbose=1, 
                                   n_jobs = -1,
                                   refit = make_scorer(rmse, greater_is_better=False))    
    
    ann_random.fit(X_train, y_train)
    
    return ann_random.best_params_

In [ ]:
ANNgrid1 = find_ann_params(H1)
ANNgrid2 = find_ann_params(H2)
ANNgrid3 = find_ann_params(H)
ANNgrid4 = find_ann_params(LR1)
ANNgrid5 = find_ann_params(KR1)
ANNgrid6 = find_ann_params(BR1)
ANNgrid7 = find_ann_params(LR2)
ANNgrid8 = find_ann_params(KR2)
ANNgrid9 = find_ann_params(BR2)
ANNgrid10 = find_ann_params(small_LR1)
ANNgrid11 = find_ann_params(small_KR1)
ANNgrid12 = find_ann_params(small_BR1)
ANNgrid13 = find_ann_params(small_LR2)
ANNgrid14 = find_ann_params(small_KR2)
ANNgrid15 = find_ann_params(small_BR2)

In [ ]:
print('----- H1 -----')
print(ANNgrid1)
print('----- H2 -----')
print(ANNgrid2)
print('----- H -----')
print(ANNgrid3)
print('----- LR1 -----')
print(ANNgrid4)
print('----- KR1 -----')
print(ANNgrid5)
print('----- BR1 -----')
print(ANNgrid6)
print('----- LR2 -----')
print(ANNgrid7)
print('----- KR2 -----')
print(ANNgrid8)
print('----- BR2 -----')
print(ANNgrid9)
print('----- small_LR1 -----')
print(ANNgrid10)
print('----- small_KR1 -----')
print(ANNgrid11)
print('----- small_BR1 -----')
print(ANNgrid12)
print('----- small_LR2 -----')
print(ANNgrid13)
print('----- small_KR2 -----')
print(ANNgrid14)
print('----- small_BR2 -----')
print(ANNgrid15)

In [ ]:
def ann(df):

    X_train, X_test, y_train, y_test = train_test(df)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform (X_test) 

   #hidden_layer_sizes = 
    activation = ['tanh', 'relu']
    batch_size = [10, 50, 100]
    learning_rate = ['constant','invscaling', 'adaptive']
    learning_rate_init = [0.0001, 0.001, 0.01]
    momentum = [0.3, 0.6, 0.9]

    params = {'activation': activation,
              'batch_size' : batch_size,
              'learning_rate' : learning_rate,
              'learning_rate_init' : learning_rate_init,
              'momentum' : momentum}    


    scorers = {'RMSE': make_scorer(rmse, greater_is_better=False),
               'MAE': make_scorer(mean_absolute_error, greater_is_better=False)}

    
    cv = RepeatedKFold(n_splits=10, n_repeats=10)
    
    random.seed(42)
    grid = GridSearchCV(estimator=MLPRegressor(solver='adam'),
                              param_grid=params, 
                              verbose=1, 
                              cv=cv, 
                              n_jobs =-1, 
                              scoring=scorers, 
                              refit = 'RMSE')
    
    grid = grid.fit(X_train, y_train)
    
    y_pred = grid.predict(X_test)
    
    print('Parameters used:', grid.best_params_,'\n')
    print('Cross Validation:')
    print('RMSE Score:', round(-1*grid.best_score_,4))
    print('MAE Score:',round(grid.cv_results_['mean_test_MAE'][grid.cv_results_['rank_test_MAE'] == 1].item(0)*-1,4))
    print ('\nHoldout set:')
    print('RMSE Score:',round(-1*grid.score(X_test, y_test),4))
    print('MAE Score:',round(mean_absolute_error(y_pred, y_test),4))

In [ ]:
ann(H1)

In [ ]:
ann(H2)

In [ ]:
ann(H)

In [ ]:
ann(LR1)

In [ ]:
ann(KR1)

In [ ]:
ann(BR1)

In [ ]:
ann(LR2)

In [ ]:
ann(KR2)

In [ ]:
ann(BR2)

In [ ]:
ann(small_LR1)

In [ ]:
ann(small_KR1)

In [ ]:
ann(small_BR1)

In [ ]:
ann(small_LR2)

In [ ]:
ann(small_KR2)

In [ ]:
ann(small_BR2)

In [ ]:
H1_baseline_score = 8.4469
H1_rf_h_score = 6.9164
H1_svr_h_score = 7.0482
H1_nn_h_score = 6.9748

H2_baseline_score = 7.8965
H2_rf_h_score = 5.7952
H2_svr_h_score = 5.1928
H2_nn_h_score = 6.3536

H_baseline_score = 9.8437
H_rf_h_score = 9.6758
H_svr_h_score = 11.481
H_nn_h_score = 12.3553



scores = {'Baseline':[H1_baseline_score, H2_baseline_score, H_baseline_score], 
          'Random Forest':[H1_rf_h_score, H2_rf_h_score, H_rf_h_score],
         'Support Vector Regression': [H1_svr_h_score, H2_svr_h_score, H_svr_h_score],
         'Neural Network': [H1_nn_h_score, H2_nn_h_score, H_nn_h_score]} 
  
# Creates pandas DataFrame. 
score_df = pd.DataFrame(scores, index =['House 1', 'House 2', 'Houses']) 

ax = score_df[['Baseline', 'Random Forest', 'Support Vector Regression', 'Neural Network']].plot(kind='bar',
              figsize=(15,7), color=['cadetblue', 'forestgreen', 'dodgerblue', 'rebeccapurple'], fontsize=13);
ax.legend(loc = 2, prop={'size': 15})
ax.set_alpha(0.8)
ax.set_title("Test score",
fontsize=18)
ax.set_ylabel("RMSE (lower is better)", fontsize=15, fontweight='bold');
ax.set_xticklabels(score_df.index,rotation=0, fontsize=15, fontweight='bold')
ax.set_ylim([0, 13])

for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x()+0.015, i.get_height()+0.1, \
            str(round((i.get_height()), 2)), fontsize=12, color='black',
                rotation=0, fontweight='bold')
    
    




In [ ]:
H1_baseline_cv_score = 6.6862
H1_rf_cv_score = 5.9731
H1_svr_cv_score = 5.9725
H1_nn_cv_score = 6.4114

H2_baseline_cv_score = 10.3454
H2_rf_cv_score = 8.489
H2_svr_cv_score = 10.0091
H2_nn_cv_score = 9.8298

H_baseline_cv_score = 8.439
H_rf_cv_score = 7.7593
H_svr_cv_score = 8.8523
H_nn_cv_score = 9.5298



cv_scores = {'Baseline':[H1_baseline_cv_score, H2_baseline_cv_score, H_baseline_cv_score], 
          'Random Forest':[H1_rf_cv_score, H2_rf_cv_score, H_rf_cv_score],
         'Support Vector Regression': [H1_svr_cv_score, H2_svr_cv_score, H_svr_cv_score],
         'Neural Network': [H1_nn_cv_score, H2_nn_cv_score, H_nn_cv_score]} 
  
# Creates pandas DataFrame. 
scorecv_df = pd.DataFrame(cv_scores, index =['House 1', 'House 2', 'Houses']) 

In [ ]:
ax = scorecv_df[['Baseline', 'Random Forest', 'Support Vector Regression', 'Neural Network']].plot(kind='bar',
              figsize=(15,7), color=['cadetblue', 'forestgreen', 'dodgerblue', 'rebeccapurple'], fontsize=13);
ax.legend(loc = 2, prop={'size': 15})
ax.set_alpha(0.8)
ax.set_title("Cross-validation score",
fontsize=18)
ax.set_ylabel("RMSE (lower is better)", fontsize=15, fontweight='bold');
ax.set_xticklabels(score_df.index,rotation=0, fontsize=15, fontweight='bold')
ax.set_ylim([0, 15])

for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x()+0.015, i.get_height()+0.1, \
            str(round((i.get_height()), 2)), fontsize=12, color='black',
                rotation=0, fontweight='bold')
    